# Optimizing the Connectivity, Robustness, and Vulnerability of Jeepney Route Networks Using Genetic Algorithm

## Setting Up

In [277]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import networkx as nx
import shapely
import folium
import geojson
import math
import osmnx as ox
from rtree import index as rtree_index
import pickle
import copy

from shapely.ops import unary_union
from shapely.geometry import Polygon, MultiPolygon, LineString, Point
from geopy.distance import geodesic

from __future__ import absolute_import, division
from math import radians, sin, cos, sqrt, atan2, exp, log
import webbrowser
import random
from scipy.spatial import KDTree
from scipy.spatial.distance import euclidean

ox.settings.log_console=True
ox.settings.use_cache=True

In [2]:
# Defining classes for the dataframes
class AmenityPoint:
    def __init__(self, geometry, lat, lon, amenity, name, addr_city):
        self.geometry = geometry
        self.lat = lat
        self.lon = lon
        self.amenity = amenity
        self.name = name
        self.addr_city = addr_city

class AmenityPolygon:
    def __init__(self, geometry, lat, lon, amenity, name, addr_city):
        self.geometry = geometry
        self.lat = lat
        self.lon = lon
        self.amenity = amenity
        self.name = name
        self.addr_city = addr_city
        
class stopCandidate:
    def __init__(self, lat, long, isTranspo, id, area):
        self.lat = lat
        self.long = long
        self.isTranspo = isTranspo
        self.enabled = False
        self.id = id #node ID
        self.area = area
        self.degree = 0
        
    def enable(self):
        self.enabled = True
        
    def disable(self):
        self.enabled = False
        
    def getLat(self):
        return self.lat
    
    def getLong(self):
        return self.long
    
    def getArea(self):
        return self.area
    
    def getDegree(self):
        return self.degree
    
class networkObj():
    def __init__(self, routes, stops, graph):
        self.routes = routes
        self.stops = stops
        self.fitness_score = 0
        self.graph = graph

In [3]:
# For units
def degrees_to_meters(angle_degrees):
    return angle_degrees * 6371000 * math.pi / 180

def meters_to_degrees(distance_meters):
    return distance_meters / 6371000 * 180 / math.pi

## Reading Data

In [4]:
# Read the building footprints data

#buildingfootprints_gdf = gpd.read_file('manila_building_footprints.geojson')

#buildingfootprints_gdf.head()

In [5]:
# READING DATA (All Amenities in Manila)
merged_amenities_points_gdf = gpd.read_file('./merged and cleaned amenities/merged_amenity_points/merged_amenities_points.shp', crs='epsg:3123')
merged_amenities_polygons_gdf = gpd.read_file('./merged and cleaned amenities/merged_amenity_polygons/cleaned_merged_with_OSM.shp', crs='epsg:3123')

merged_amenities_polygons_gdf['amenity_points'] = None

In [6]:
# OLD DATA (FOR FAST TESTING)
# Load the Manila amenities data into a Geopandas dataframe
from shapely import wkt

manila_amenities_df = pd.read_csv('manila_amenities.csv')
manila_amenities_df['geometry'] = manila_amenities_df['geometry'].apply(wkt.loads)
manila_amenities_gdf = gpd.GeoDataFrame(manila_amenities_df, crs='epsg:3123')

# Separate into point and polygon dataframes
manila_amenities_polygon_gdf = manila_amenities_gdf[manila_amenities_gdf['geometry'].geom_type == 'Polygon']
manila_amenities_point_gdf = manila_amenities_gdf[manila_amenities_gdf['geometry'].geom_type == 'Point']
manila_amenities_multipoly_gdf = manila_amenities_gdf[manila_amenities_gdf['geometry'].geom_type == 'MultiPolygon']

# Append multipolygons to the polygon dataframe
manila_amenities_polygon_gdf = gpd.GeoDataFrame(pd.concat([manila_amenities_polygon_gdf, manila_amenities_multipoly_gdf], ignore_index=True))

# Reset point dataframe index
manila_amenities_point_gdf.reset_index(drop=True, inplace=True)

# Add a column to the polygon dataframe to store a list of Amenity Points within the polygon
manila_amenities_polygon_gdf['amenity_points'] = None

In [7]:
# For each polygon in the polygon dataframe, find all the points from the point dataframe lying inside that polygon
# Store the list of points in the 'amenity_points' column of the polygon dataframe as a list of point indices

# Create spatial index for points
idx = rtree_index.Index()
for j, point in manila_amenities_point_gdf.iterrows():
    idx.insert(j, point['geometry'].bounds)

# Iterate over polygons
for i, polygon in manila_amenities_polygon_gdf.iterrows():
    points_within_polygon = []
    
    # Iterate over points within the bounding box of the polygon
    for j in idx.intersection(polygon['geometry'].bounds):
        point = manila_amenities_point_gdf.loc[j]
        if polygon['geometry'].intersects(point['geometry']):
            points_within_polygon.append(j)
    
    manila_amenities_polygon_gdf.at[i, 'amenity_points'] = points_within_polygon

In [8]:
manila_amenities_polygon_gdf['amenity'].unique()

array(['education', 'finance', 'government offices', 'grocery', 'health',
       'malls', 'residential areas', 'security'], dtype=object)

In [9]:
manila_amenities_point_gdf['amenity'].unique()

array(['education', 'finance', 'government offices', nan, 'public_market',
       'health', 'malls', 'residential areas', 'security',
       'transportation'], dtype=object)

### Reading Population data

In [10]:
# Reading population data
manila_population_df = pd.read_csv('manila-population-polygon.csv')
manila_population_df['geometry'] = manila_population_df['geometry'].apply(wkt.loads)
manila_population_gdf = gpd.GeoDataFrame(manila_population_df, crs='epsg:3123')

# Create a base map centered around Manila
map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

# Add points to the map
for index, row in manila_population_gdf.iterrows():
    folium.CircleMarker(location=[row['latitude'], row['longitude']],
                        radius=1,  # Adjust the radius as needed for population density representation
                        color='blue',  # Change color as needed
                        fill=True,
                        fill_color='blue'  # Change fill color as needed
                        ).add_to(m)
    
m.save('population.html') # Save the map to an HTML file

## Initial City Network Creation

In [11]:
# GENERATION OF MAIN CITY GRAPH
# IF FIRST TIME RUNNING, RUN THIS CODE TO GENERATE THE GRAPH
def generate_graph():
    place = 'Manila, Philippines'
    mode = 'drive'
    graph = ox.graph_from_place(place, network_type = mode) # Generate graph of Metro manila
    ox.save_graphml(graph, 'map/Manila.graphml') # Save it as a file

def load_graph():
    graph = ox.load_graphml('map/Manila.graphml')
    
    print("Graph loaded successfully")
    print("NUMBER OF EDGES: ", graph.number_of_edges())
    print("NUMBER OF NODES: ", graph.number_of_nodes())
    print('\n')
    return graph


# THIS IS THE MAIN GRAPH FOR THE CITY TO BE USED FOR ALL FUNCTIONS
CITY_GRAPH = load_graph()

Graph loaded successfully
NUMBER OF EDGES:  12617
NUMBER OF NODES:  4926




In [12]:
# Buckle up. We're trying to create a network out of this monstrosity of a dataframe
# Create a networkx graph

def create_network(amenities_polygon_gdf, amenities_point_gdf):
    amenities_network = nx.Graph()

   # Add polygon nodes
    for index, row in amenities_polygon_gdf.iterrows():
        # Check if essential columns exist in the row
        if 'geometry' in row and 'amenity' in row and 'name' in row and 'addr_city' in row and 'amenity_points' in row:
            # Generate a unique node identifier for polygons
            node_id = f"polygon_{index}"
            amenities_network.add_node(node_id, polygon_index=index, geometry=row['geometry'], lat=row['geometry'].centroid.y, lon=row['geometry'].centroid.x, amenity=row['amenity'], name=row.get('name', ''), addr_city=row['addr_city'], amenity_points=row['amenity_points'])
        else:
            print(f"Skipping row {index} in amenities_polygon_gdf due to missing data.")

    # Add point nodes
    for index, row in amenities_point_gdf.iterrows():
        # Check if essential columns exist in the row
        if 'geometry' in row and 'amenity' in row and 'name' in row and 'addr_city' in row:
            # Generate a unique node identifier for points
            node_id = f"point_{index}"
            
            # This part checks whether the point is a transportation or not 
            if row['amenity'] == 'transportation':
                isTranspo = True
            else:
                isTranspo = False
            amenities_network.add_node(node_id, point_index=index, geometry=row['geometry'], lat=row['y'], lon=row['x'], amenity=row['amenity'], name=row.get('name', ''), addr_city=row['addr_city'], is_in_polygon=False, isTranspo = isTranspo)
        else:
            print(f"Skipping row {index} in amenities_point_gdf due to missing data.")
            
    return amenities_network

### For Filtering the roads and other features

In [13]:
# GETTING ROADS AND WATERWAYS

# Get all the roads in Manila
manila_road = ox.graph_to_gdfs(CITY_GRAPH,nodes=False, edges=True)


# Get all the roads that are not junctions (ex. Roundabouts, intersection, etc.)
filtered_roads = manila_road[manila_road['junction'].isna()]

# Separate roads whose widths are only one value and those that are more than 1 (lists)
rows_with_lists = filtered_roads[filtered_roads['highway'].apply(lambda x: isinstance(x, list))]
rows_with_strings = filtered_roads[filtered_roads['highway'].apply(lambda x: isinstance(x, str))]

filter_options = ['primary', 'secondary', 'tertiary', 'trunk', 'unclassified']
separation_options = ['primary', 'secondary', 'tertiary', 'unclassified']

# Get the roads whose widths are above the threshold
def check_list(lst):
    return any(x in filter_options for x in lst)

# Download OpenStreetMap data for the area of interest
waterways = ox.features_from_place('Manila, Philippines', tags={'waterway': True})
filtered_rivers = waterways[waterways['waterway'].isin(['river'])]
filtered_streams = waterways[waterways['waterway'].isin(['stream'])]

# Get all the roads with the allowed road types
filtered_roads_strings = rows_with_strings.loc[rows_with_strings['highway'].isin(filter_options)] 
filtered_roads_lists = rows_with_lists[rows_with_lists['highway'].apply(check_list)]

### Visualization of Roads and Rivers

In [14]:
#Plotting filtered roads FOR VISUALIZATION ONLY
def plot_all_filtered_roads():
    map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
    m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

    # Iterate through each road
    for index, road in filtered_roads_strings.iterrows():
        line_coords = list(road['geometry'].coords)

        if road['highway'] == 'primary':
            folium.PolyLine(locations=[(y, x) for x, y in line_coords], color='blue').add_to(m)

        if road['highway'] == 'secondary':
            folium.PolyLine(locations=[(y, x) for x, y in line_coords], color='red').add_to(m)

        if road['highway'] == 'tertiary':
            folium.PolyLine(locations=[(y, x) for x, y in line_coords], color='green').add_to(m)

        if road['highway'] == 'unclassified':
            folium.PolyLine(locations=[(y, x) for x, y in line_coords], color='orange').add_to(m)

    for index, road in filtered_roads_lists.iterrows():
        line_coords = list(road['geometry'].coords)
        folium.PolyLine(locations=[(y, x) for x, y in line_coords], color='purple').add_to(m)

    # Return the map
    return m

def plot_all_roads():
    map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
    m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

    # Iterate through each road
    for index, road in manila_road.iterrows():
        line_coords = list(road['geometry'].coords)
        folium.PolyLine(locations=[(y, x) for x, y in line_coords], color='blue').add_to(m)

    # Return the map
    return m

# TEMPORARY TO BE REMOVED
def plot_private_roads():
    map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
    m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

    # Iterate through each road
    for index, road in manila_private.iterrows():
        line_coords = list(road['geometry'].coords)
        folium.PolyLine(locations=[(y, x) for x, y in line_coords], color='blue').add_to(m)

    # Return the map
    return m

# TEMPORARY TO BE REMOVED
def plot_walk():
    map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
    m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

    # Iterate through each road
    for index, road in manila_walk.iterrows():
        line_coords = list(road['geometry'].coords)
        folium.PolyLine(locations=[(y, x) for x, y in line_coords], color='blue').add_to(m)

    # Return the map
    return m
    
# TEMPORARY TO BE REMOVED
def plot_bike():
    map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
    m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

    # Iterate through each road
    for index, road in manila_bike.iterrows():
        line_coords = list(road['geometry'].coords)
        folium.PolyLine(locations=[(y, x) for x, y in line_coords], color='blue').add_to(m)

    # Return the map
    return m

## For Amenity and Zone Connection

In [15]:
# This function will find which road or river intersects between amenities
# Create spatial index
filtered_roads_strings_sindex = filtered_roads_strings.sindex
filtered_roads_lists_sindex = filtered_roads_lists.sindex
filtered_rivers_sindex = filtered_rivers.sindex
filtered_streams_sindex = filtered_streams.sindex

def find_intersecting_features(line):
    # Check intersection with filtered roads
    possible_matches_roads = filtered_roads_strings.iloc[list(filtered_roads_strings_sindex.intersection(line.bounds))]
    for index, row in possible_matches_roads.iterrows():
        if line.intersects(row['geometry']) and row['highway'] in separation_options:
            return True

    possible_matches_lists = filtered_roads_lists.iloc[list(filtered_roads_lists_sindex.intersection(line.bounds))]
    for index, row in possible_matches_lists.iterrows():
        if line.intersects(row['geometry']):
            list_highway = row['highway']
            if any(x in separation_options for x in list_highway):
                return True
    
    # Check intersection with filtered rivers
    possible_matches_rivers = filtered_rivers.iloc[list(filtered_rivers_sindex.intersection(line.bounds))]
    for index, row in possible_matches_rivers.iterrows():
        if line.intersects(row['geometry']):
            return True

    # Check intersection with filtered streams
    possible_matches_streams = filtered_streams.iloc[list(filtered_streams_sindex.intersection(line.bounds))]
    for index, row in possible_matches_streams.iterrows():
        if line.intersects(row['geometry']):
            return True
    
    return False

In [16]:
# ADD POINTS TO NX GRAPH
# Function to add only points to the networkX graph
# The other functions focuses on adding polygons, this function just iterates and adds points

def add_points_to_graph(graph, graph_to_add):
    for node_key, node_data in graph.nodes.items():
        if 'geometry' in node_data and node_data['geometry'].geom_type in ['Point']:   
            graph_to_add.add_node(node_key, geometry=node_data['geometry'], name=node_data['name'], lat=node_data['lat'], amenity=node_data['amenity'],
                                lon=node_data['lon'])
                
                

In [17]:
# LEVEL 1 - COMBINE AMENITIES BY POLYGON
# Creates a copy of a graph and connects non-contiguous and non-overlapping shapes instead of merging

def combine_amenities_by_polygon(graph, max_distance, max_perimeter):
    combined_graph = nx.Graph()
    list_to_merge = []
    idx = rtree_index.Index()
    
    # Iterates through each polygon and then enlarges and gets the intersecting ones for easier iteration later
    for node_key, node_data in graph.nodes.items():
        # Ensure that the bounding box coordinates are passed as a tuple
        if 'geometry' in node_data and node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
            enlarged_polygon = node_data['geometry'].buffer(meters_to_degrees(max_distance))
            bounds = enlarged_polygon.bounds
            bounds_float = tuple(float(coord) for coord in bounds)
            numeric_key = int(node_key.split('_')[1])
            idx.insert(numeric_key, bounds_float)
    
    #Iterating through each polygon
    for node_key, node_data in list(graph.nodes.items()):
        if 'geometry' in node_data and node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
            nodes_to_merge = []
            
            #Check first if it is already in a list of polygons to be merged
            for merge_list in list_to_merge:
                if node_key in merge_list:
                    nodes_to_merge = merge_list
                    break
            
            # If this is a new node that is not part of any list, add itself to the list for merging later
            if not nodes_to_merge:
                nodes_to_merge.append(node_key)
            
            # Distance 
            total_distance = 0 # This is to calculate the total distance
            combined_node = graph.nodes[node_key]['geometry']
            
            # Then iterate through other polygons that intersect that polygon based on bounds
            for other_node_key in idx.intersection(node_data['geometry'].bounds):
                formatted_key = f"polygon_{other_node_key}"
                other_node_data = graph.nodes[formatted_key]
                if 'geometry' in other_node_data and other_node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
                    # Check if its not the same node, it is the same amenity, and is not already in the list to merge
                    if node_key != formatted_key and node_data['amenity'] == other_node_data['amenity']:
                        distance = degrees_to_meters(node_data['geometry'].distance(other_node_data['geometry']))

                        if distance <= max_distance:
                            line_between_centroids = LineString([node_data['geometry'].centroid, other_node_data['geometry'].centroid])
                            amenities_intersecting = any(graph.nodes[amenity_key]['geometry'].intersects(line_between_centroids) for amenity_key in graph.nodes if amenity_key != node_key and amenity_key != formatted_key and graph.nodes[amenity_key]['amenity'] != node_data['amenity'])
                            
                            # Check if it does not exceed the max perimeter
                            combined_node = shapely.ops.unary_union([combined_node, graph.nodes[formatted_key]['geometry']])
                            total_distance += degrees_to_meters(combined_node.length)
                            
                            if not amenities_intersecting and total_distance < max_perimeter and not find_intersecting_features(line_between_centroids):
                                nodes_to_merge.append(formatted_key)
            
            if nodes_to_merge not in list_to_merge:
                list_to_merge.append(nodes_to_merge) # Add to the list to merge the polygons later
                
            
                
    temp_graph = to_graph(list_to_merge)
    lists = graph_to_list(temp_graph)


    # Now we will finally connect all polygons in the list
    for merge_list in lists:
        first = True
        for node_key in merge_list:
            if first:
                combined_node = graph.nodes[node_key]['geometry']
                combined_node_amenity = graph.nodes[node_key]['amenity']
                combined_node_key = node_key
                combined_node_geometry = graph.nodes[node_key]['geometry']
                combined_node_name = graph.nodes[node_key]['name']
                combined_node_lat = graph.nodes[node_key]['lat']
                combined_node_lon = graph.nodes[node_key]['lon']
                combined_node_points = graph.nodes[node_key]['amenity_points']
                first = False
            else:
                combined_node = shapely.ops.unary_union([combined_node_geometry, graph.nodes[node_key]['geometry']])
                combined_node_geometry = combined_node
                combined_node_name = combine_names(combined_node_name, graph.nodes[node_key].get('name'))
                combined_node_lat = combined_node_geometry.centroid.x
                combined_node_lon = combined_node_geometry.centroid.x
                combined_node_points += graph.nodes[node_key].get('amenity_points', 0)
                
        combined_graph.add_node(combined_node_key, geometry=combined_node_geometry, name=combined_node_name, lat=combined_node_lat, amenity=combined_node_amenity,
                                lon=combined_node_lon, amenity_points=combined_node_points)

    return combined_graph

# TEMPORARY SOLUTION FOR NULL NAMES
def combine_names(name1, name2):
    # Combine names ensuring that no null values are included
    if isinstance(name1, str) and isinstance(name2, str):
        return f"{name1}, {name2}"
    elif isinstance(name1, str):
        return name1
    elif isinstance(name2, str):
        return name2
    else:
        return None

In [18]:
# TEMPORARY

def combine_residential(graph, max_distance, max_perimeter):
    combined_graph = nx.Graph()
    list_to_merge = []
    idx = rtree_index.Index()
    
    for node_key, node_data in graph.nodes.items():
        if 'amenity' in node_data and node_data['amenity'] == 'residential areas':
            if 'geometry' in node_data and node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
                enlarged_polygon = node_data['geometry'].buffer(meters_to_degrees(max_distance))
                bounds = enlarged_polygon.bounds
                bounds_float = tuple(float(coord) for coord in bounds)
                numeric_key = int(node_key.split('_')[1])
                idx.insert(numeric_key, bounds_float)

    loop_count = 1
    for node_key, node_data in graph.nodes.items():
        print("Polgon Count: ", loop_count, " / ", total_residential)
        
        if 'amenity' in node_data and node_data['amenity'] == 'residential areas':
            if 'geometry' in node_data and node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
                nodes_to_merge = []
            
                #Check first if it is already in a list of polygons to be merged
                for merge_list in list_to_merge:
                    if node_key in merge_list:
                        nodes_to_merge = merge_list
                        break
            
                # If this is a new node that is not part of any list, add itself to the list for merging later
                if not nodes_to_merge:
                    nodes_to_merge.append(node_key)
            
                # Distance 
                total_distance = 0 # This is to calculate the total distance
                combined_node = graph.nodes[node_key]['geometry']

                sub_poly_count = 1
                for other_node_key in idx.intersection(node_data['geometry'].bounds):
                    print(f"Currently checking {sub_poly_count}")
                    
                    formatted_key = f"polygon_{other_node_key}"
                    other_node_data = graph.nodes[formatted_key]
                    if 'amenity' in other_node_data and other_node_data['amenity'] == 'residential areas':
                        if 'geometry' in other_node_data and other_node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
                            # Check if its not the same node, it is the same amenity, and is not already in the list to merge
                            if node_key != formatted_key and node_data['amenity'] == other_node_data['amenity']:
                                distance = degrees_to_meters(node_data['geometry'].distance(other_node_data['geometry']))
                                line_between_centroids = LineString([node_data['geometry'].centroid, other_node_data['geometry'].centroid])
                                amenities_intersecting = any(graph.nodes[amenity_key]['geometry'].intersects(line_between_centroids) for amenity_key in graph.nodes if amenity_key != node_key and amenity_key != formatted_key and graph.nodes[amenity_key]['amenity'] != node_data['amenity'])
                        
                                # Check if it does not exceed the max perimeter
                                combined_node = shapely.ops.unary_union([combined_node, graph.nodes[formatted_key]['geometry']])
                                total_distance += degrees_to_meters(combined_node.length)
                        
                                if not amenities_intersecting and total_distance < max_perimeter and not find_intersecting_features(line_between_centroids):
                                    nodes_to_merge.append(formatted_key)
                    sub_poly_count += 1
            
                if nodes_to_merge not in list_to_merge:
                    list_to_merge.append(nodes_to_merge) # Add to the list to merge the polygons later
                    
        loop_count += 1
    temp_graph = to_graph(list_to_merge)
    lists = graph_to_list(temp_graph)


    # Now we will finally connect all polygons in the list
    for merge_list in lists:
        first = True
        for node_key in merge_list:
            if first:
                combined_node = graph.nodes[node_key]['geometry']
                combined_node_amenity = graph.nodes[node_key]['amenity']
                combined_node_key = node_key
                combined_node_geometry = graph.nodes[node_key]['geometry']
                combined_node_name = graph.nodes[node_key]['name']
                combined_node_lat = graph.nodes[node_key]['lat']
                combined_node_lon = graph.nodes[node_key]['lon']
                combined_node_points = graph.nodes[node_key]['amenity_points']
                first = False
            else:
                combined_node = shapely.ops.unary_union([combined_node_geometry, graph.nodes[node_key]['geometry']])
                combined_node_geometry = combined_node
                combined_node_name = combine_names(combined_node_name, graph.nodes[node_key].get('name'))
                combined_node_lat = combined_node_geometry.centroid.x
                combined_node_lon = combined_node_geometry.centroid.x
                combined_node_points += graph.nodes[node_key].get('amenity_points', 0)
        
        combined_graph.add_node(combined_node_key, geometry=combined_node_geometry, name=combined_node_name, lat=combined_node_lat, amenity=combined_node_amenity,
                                lon=combined_node_lon, amenity_points=combined_node_points)

    return combined_graph

In [19]:
# TEMPORARY
def combine_small_residential(graph, max_distance, max_perimeter):
    combined_graph = nx.Graph()
    list_to_merge = []
    idx = rtree_index.Index()
    
    for node_key, node_data in graph.nodes.items():
        if 'amenity' in node_data and node_data['amenity'] == 'residential areas':
            if 'geometry' in node_data and node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
                enlarged_polygon = node_data['geometry'].buffer(meters_to_degrees(20))
                bounds = enlarged_polygon.bounds
                bounds_float = tuple(float(coord) for coord in bounds)
                numeric_key = int(node_key.split('_')[1])
                idx.insert(numeric_key, bounds_float)

    loop_count = 1
    for node_key, node_data in graph.nodes.items():
        print("Polgon Count: ", loop_count, " / ", total_residential)
        
        if 'amenity' in node_data and node_data['amenity'] == 'residential areas':
            if 'geometry' in node_data and node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
                nodes_to_merge = []
            
                #Check first if it is already in a list of polygons to be merged
                for merge_list in list_to_merge:
                    if node_key in merge_list:
                        nodes_to_merge = merge_list
                        break
            
                # If this is a new node that is not part of any list, add itself to the list for merging later
                if not nodes_to_merge:
                    nodes_to_merge.append(node_key)
            
                # Distance 
                total_distance = 0 # This is to calculate the total distance
                combined_node = graph.nodes[node_key]['geometry']

                sub_poly_count = 1
                for other_node_key in idx.intersection(node_data['geometry'].bounds):
                    print(f"Polygon {loop_count}: Currently checking {sub_poly_count}")
                    
                    formatted_key = f"polygon_{other_node_key}"
                    other_node_data = graph.nodes[formatted_key]
                    if 'amenity' in other_node_data and other_node_data['amenity'] == 'residential areas':
                        if 'geometry' in other_node_data and other_node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
                            # Check if its not the same node, it is the same amenity, and is not already in the list to merge
                            if node_key != formatted_key and node_data['amenity'] == other_node_data['amenity']:
                                distance = degrees_to_meters(node_data['geometry'].distance(other_node_data['geometry']))
                                line_between_centroids = LineString([node_data['geometry'].centroid, other_node_data['geometry'].centroid])
                                amenities_intersecting = any(graph.nodes[amenity_key]['geometry'].intersects(line_between_centroids) for amenity_key in graph.nodes if amenity_key != node_key and amenity_key != formatted_key and graph.nodes[amenity_key]['amenity'] != node_data['amenity'])
                        
                                # Check if it does not exceed the max perimeter
                                combined_node = shapely.ops.unary_union([combined_node, graph.nodes[formatted_key]['geometry']])
                                total_distance += degrees_to_meters(combined_node.length)
                        
                                if not amenities_intersecting and total_distance < max_perimeter and not find_intersecting_features(line_between_centroids):
                                    nodes_to_merge.append(formatted_key)
                    sub_poly_count += 1
            
                if nodes_to_merge not in list_to_merge:
                    list_to_merge.append(nodes_to_merge) # Add to the list to merge the polygons later
                    
        loop_count += 1
        
    temp_graph = to_graph(list_to_merge)
    lists = graph_to_list(temp_graph)


    # Now we will finally connect all polygons in the list
    for merge_list in lists:
        first = True
        for node_key in merge_list:
            if first:
                combined_node = graph.nodes[node_key]['geometry']
                combined_node_amenity = graph.nodes[node_key]['amenity']
                combined_node_key = node_key
                combined_node_geometry = graph.nodes[node_key]['geometry']
                combined_node_name = graph.nodes[node_key]['name']
                combined_node_lat = graph.nodes[node_key]['lat']
                combined_node_lon = graph.nodes[node_key]['lon']
                combined_node_points = graph.nodes[node_key]['amenity_points']
                first = False
            else:
                combined_node = shapely.ops.unary_union([combined_node_geometry, graph.nodes[node_key]['geometry']])
                combined_node_geometry = combined_node
                combined_node_name = combine_names(combined_node_name, graph.nodes[node_key].get('name'))
                combined_node_lat = combined_node_geometry.centroid.x
                combined_node_lon = combined_node_geometry.centroid.x
                combined_node_points += graph.nodes[node_key].get('amenity_points', 0)
        
        print("Added")
        combined_graph.add_node(combined_node_key, geometry=combined_node_geometry, name=combined_node_name, lat=combined_node_lat, amenity=combined_node_amenity,
                                lon=combined_node_lon, amenity_points=combined_node_points)

    return combined_graph

In [20]:
# To merge duplicates in lists
def to_graph(nodes):
    G = nx.Graph()
    for part in nodes:
        G.add_nodes_from(part)
        G.add_edges_from(to_edges(part))
    return G

def to_edges(nodes):
    it = iter(nodes)
    last = next(it)

    for current in it:
        yield last, current
        last = current
        
def graph_to_list(G):
    connected_components = nx.connected_components(G)
    lists = [list(component) for component in connected_components]
    return lists

### Visualization

In [21]:
# 1 - Function to plot/visualize main graph network on the map
def plot_network_on_map(amenities_network, initial_location=[0, 0], zoom_start=10):
    # Create a map centered at the initial location
    map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
    m = folium.Map(location=map_center, zoom_start=zoom_start, tiles='openstreetmap')
    
    #Colours for Visualization
    amenity_colors = {
        'education': 'green',
        'finance': 'blue',
        'government offices': 'red',
        'grocery': 'orange',
        'health': 'magenta',
        'malls': 'yellow',
        'residential areas': 'brown',
        'security': 'gray',
        'transportation': 'lightblue',
        'others': 'black'
    }

    # Iterate over the nodes in the network
    for node, data in amenities_network.nodes(data=True):
        # Check if the node has a geometry attribute
        if 'geometry' in data:
            # Get the geometry of the node
            geometry = data['geometry']

            # Check the geometry type and plot accordingly
            if geometry.geom_type == 'Point':
                # Plot a marker for points    
                #folium.Marker(location=[geometry.y, geometry.x], popup=f"{data['name']}").add_to(m)
                continue
            elif geometry.geom_type in ['Polygon', 'MultiPolygon']:
                # Plot polygons or multipolygons
                color = amenity_colors[data.get('amenity')]
                if geometry.geom_type == 'Polygon':
                    polygons = [geometry]
                else:
                    polygons = geometry.geoms

                for polygon in polygons:
                    coordinates = []
                    for point in polygon.exterior.coords:
                        coordinates.append([point[1], point[0]])
                    folium.Polygon(locations=coordinates, fill=True, color=color, fill_opacity=0.4).add_to(m)

    # Return the map
    return m


In [22]:
# LEVEL 2 - Function to connect zones in a network
def create_zone_network(graph, max_distance):
    connect_graph = nx.Graph()
    network_id = 1
    list_to_connect = []
    idx = rtree_index.Index()
    
    for node_key, node_data in graph.nodes.items():
        enlarged_polygon = node_data['geometry'].buffer(meters_to_degrees(max_distance))
        bounds = enlarged_polygon.bounds
        bounds_float = tuple(float(coord) for coord in bounds)
        numeric_key = int(node_key.split('_')[1])
        idx.insert(numeric_key, bounds_float)
    
    for node_key, node_data in list(graph.nodes.items()):
        if 'geometry' in node_data and node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
            connect_nodes = []
            
            #Check first if it is already in a list of polygons to be connected
            for connect_list in list_to_connect:
                if node_key in connect_list:
                    connect_nodes = connect_list
                    break
            
            # If this is a new node that is not part of any list, add itself to the list for merging later
            if not connect_nodes:
                connect_nodes.append(node_key)
                
            # If this is not a residential area that is its own zone
            if node_key not in pop_graph or not pop_graph.nodes[node_key]['is_a_zone']:
                for other_node_key in idx.intersection(node_data['geometry'].bounds):
                    formatted_key = f"polygon_{other_node_key}"
                    other_node_data = graph.nodes[formatted_key]
                    if 'geometry' in other_node_data and other_node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
                        if node_key != formatted_key:

                            distance = degrees_to_meters(node_data['geometry'].distance(other_node_data['geometry']))

                            # Check if they are within distance of each other
                            if distance <= max_distance:
                                line_between_centroids = LineString([node_data['geometry'].centroid, other_node_data['geometry'].centroid])
                                if not find_intersecting_features(line_between_centroids):
                                    if formatted_key not in pop_graph or not pop_graph.nodes[formatted_key]['is_a_zone']:
                                        connect_nodes.append(formatted_key)
                                
            if connect_nodes not in list_to_connect:
                list_to_connect.append(connect_nodes) # Add to the list to merge the polygons later
                
    temp_graph = to_graph(list_to_connect)
    lists = graph_to_list(temp_graph)

    # Now we will finally connect all polygons in the list
    for merge_list in lists:
        first = True
        for node_key in merge_list:
            if first:
                combined_node = graph.nodes[node_key]['geometry']
                combined_node_amenity = graph.nodes[node_key]['amenity']
                combined_node_key = node_key
                combined_node_geometry = graph.nodes[node_key]['geometry']
                combined_node_name = graph.nodes[node_key]['name']
                combined_node_lat = graph.nodes[node_key]['lat']
                combined_node_lon = graph.nodes[node_key]['lon']
                combined_node_points = graph.nodes[node_key]['amenity_points']
                first = False
            else:
                combined_node = shapely.ops.unary_union([combined_node_geometry, graph.nodes[node_key]['geometry']])
                combined_node_geometry = combined_node
                combined_node_name = combine_names(combined_node_name, graph.nodes[node_key].get('name'))
                combined_node_lat = combined_node_geometry.centroid.x
                combined_node_lon = combined_node_geometry.centroid.x
                combined_node_points += graph.nodes[node_key].get('amenity_points', 0)
                
        network_id += 1
        connect_graph.add_node(combined_node_key, geometry=combined_node_geometry, name=combined_node_name, lat=combined_node_lat, amenity=combined_node_amenity,
                                lon=combined_node_lon, amenity_points=combined_node_points, network_id=network_id)

    return connect_graph

# TEMPORARY SOLUTION FOR NULL NAMES
def combine_names(name1, name2):
    # Combine names ensuring that no null values are included
    if isinstance(name1, str) and isinstance(name2, str):
        return f"{name1}, {name2}"
    elif isinstance(name1, str):
        return name1
    elif isinstance(name2, str):
        return name2
    else:
        return None

In [23]:
# 2 - Function to plot/visualize connected zones on the map
import random

# This is to better visualize the networks
def plot_connected_zones_network_on_map(graph, initial_location=[0, 0], zoom_start=10):
    # Create a map centered at the initial location
    map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
    m = folium.Map(location=map_center, zoom_start=zoom_start, tiles='openstreetmap')
    
    #Colours for Visualization
    colors = [
    "Red", "Green", "Blue", "Yellow", "Orange", "Purple", "Cyan", "Magenta", "Maroon",
    "Olive", "Lime", "Teal", "Navy", "Aqua", "Fuchsia", "Coral", "Indigo", "Violet"]
    
    color_map = {}

    # Iterate over the nodes in the network
    for node, data in graph.nodes(data=True):
        # Check if the node has a geometry attribute
        if 'geometry' in data:
            # Get the geometry of the node
            geometry = data['geometry']

            # Check the geometry type and plot accordingly
            if geometry.geom_type == 'Point':
                # Plot a marker for points    
                #folium.Marker(location=[geometry.y, geometry.x], popup=f"{data['name']}").add_to(m)
                continue
            elif geometry.geom_type in ['Polygon', 'MultiPolygon']:
                
                network_id = data["network_id"]
                
                if network_id not in color_map:
                    color = random.choice(colors)
                    color_map[network_id] = color
                else:
                    color = color_map[network_id]
                
                if geometry.geom_type == 'Polygon':
                    polygons = [geometry]
                else:
                    polygons = geometry.geoms

                for polygon in polygons:
                    coordinates = []
                    for point in polygon.exterior.coords:
                        coordinates.append([point[1], point[0]])
                    folium.Polygon(locations=coordinates, fill=True, color=color, fill_opacity=0.4).add_to(m)

    # Return the map
    return m


In [24]:
# This is to visualize the stops
def plot_stops_on_map(network_map, stops, initial_location=[0, 0], zoom_start=10):
    # Iterate over the nodes in the network
    for stop in stops:
        folium.Marker(location=[stop.lat, stop.long], popup=f"{stop.isTranspo}").add_to(network_map)
        
    return network_map

In [25]:
# Function to check population density - To be used for the zone connection
# Uses the combined graph
# Formula: Population Density = Total Population / Total Area

def check_residential_population_density(graph, threshold):
    # Create an R-tree index for efficient spatial querying
    idx = rtree_index.Index()
    
    # Populate the R-tree index with points
    for index, row in manila_population_gdf.iterrows():
        idx.insert(index, (row['longitude'], row['latitude'], row['longitude'], row['latitude']))
    
    pop_graph = nx.Graph()
    
    for node_key, node_data in graph.nodes.items():
        # Check if its a polygon and is a residential area
        if 'geometry' in node_data and node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon'] and node_data['amenity'] == "residential areas":
            total_pop = 0
            
            # Query the R-tree index to find points within the polygon
            for point_idx in idx.intersection(node_data['geometry'].bounds):
                point = manila_population_gdf.loc[point_idx]
                if Point(point['longitude'], point['latitude']).within(node_data['geometry']):
                    total_pop += point['phl_general_2020']  # Add the density
            
            density = total_pop / node_data['geometry'].area
            
            if density > threshold:
                node_data["is_a_zone"] = True
            else:
                node_data["is_a_zone"] = False
            
            pop_graph.add_node(node_key, density=density, **node_data)
    return pop_graph

In [26]:
# 2.5 - Function to plot/visualize residential areas based on population density on the map
# This is to better visualize which residential areas can become zones
def plot_population_zones_map(graph, initial_location=[0, 0], zoom_start=10):
    # Create a map centered at the initial location
    map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
    m = folium.Map(location=map_center, zoom_start=zoom_start, tiles='openstreetmap')

    # Iterate over the nodes in the network
    for node, data in graph.nodes(data=True):
        # Check if the node has a geometry attribute
        if 'geometry' in data:
            # Get the geometry of the node
            geometry = data['geometry']

            # Check the geometry type and plot accordingly
            if geometry.geom_type == 'Point':
                # Plot a marker for points    
                #folium.Marker(location=[geometry.y, geometry.x], popup=f"{data['name']}").add_to(m)
                continue
            elif geometry.geom_type in ['Polygon', 'MultiPolygon']:
                if geometry.geom_type == 'Polygon':
                    polygons = [geometry]
                else:
                    polygons = geometry.geoms

                for polygon in polygons:
                    coordinates = []
                    for point in polygon.exterior.coords:
                        coordinates.append([point[1], point[0]])
                    
                    if (data['is_a_zone']):
                        folium.Polygon(locations=coordinates, fill=True, color="green", fill_opacity=0.4).add_to(m)
                    else:
                        folium.Polygon(locations=coordinates, fill=True, color="red", fill_opacity=0.4).add_to(m)

    # Return the map
    return m

## Stop Placement

In [27]:

# CREATING STOPS
# It should return a list of coordinates/nodes for stop and a graph of stops
# if residential area, check if the population density

# Global Variables used:
# list_of_stops - List of stops
# graph_of_stops - graph of all stops placed
# CITY_GRAPH - graph of the city road networks
import random

def place_stops_on_roads(amenity_graph):    
    for node_key, node_data in amenity_graph.nodes(data=True):
        # All tranportation points are automatically stops
        if node_data['geometry'].geom_type in ['Point']:
            if node_data['amenity'] == 'transportation':
                nearest_node = ox.distance.nearest_nodes(CITY_GRAPH, node_data['lon'], node_data['lat'])
                
                # If there is an existing node in the main graph, then add it to the list and the stop graph
                if nearest_node is not None:
                    lon = CITY_GRAPH.nodes[nearest_node]['x']
                    lat = CITY_GRAPH.nodes[nearest_node]['y']
                    isTranspo = True
                    graph_of_stops.add_node(nearest_node, lon=lon, lat=lat, isTranspo=isTranspo)
                    list_of_stops.append(stopCandidate(CITY_GRAPH.nodes[nearest_node]['y'], CITY_GRAPH.nodes[nearest_node]['x'], True, nearest_node, 0))
        
        else:
            # Calculate the number of stops based on node size and population density
            num_stops, node_size = calculate_num_stops(node_key, node_data)
            
            buffer_poly = node_data['geometry'].buffer(meters_to_degrees(30))
            # Get the roads surrounding and inside the node polygons
            relevant_edges = get_relevant_edges(buffer_poly)
            
            # Place stops randomly on these roads
            place_stops_along_edges(relevant_edges, buffer_poly, num_stops, node_size)

def calculate_num_stops(node_key, node_data):
    # Example calculation based on node size and population density
    node_size = degrees_to_meters(node_data['geometry'].area) # Size of the node polygon
    # Adjust factors and formula as needed
    num_stops = 0
    
    if node_key in pop_graph:
        pop_density = pop_graph.nodes[node_key]['density']
        num_stops = node_size * pop_density / 10000000  # Adjust this factor as needed
        
        if num_stops < 1:
            num_stops = 1
        elif num_stops > 3:
            num_stops = 3
    else:
        list_sum = len(node_data['amenity_points'])

        if list_sum > 0:
            num_stops = 1
        
    return int(num_stops), node_size


# Create spatial index
filtered_roads_strings_sindex = filtered_roads_strings.sindex
filtered_roads_lists_sindex = filtered_roads_lists.sindex
def get_relevant_edges(polygon):
    relevant_edges = []
    
    # Check intersection with filtered roads
    possible_matches_roads = filtered_roads_strings.iloc[list(filtered_roads_strings_sindex.intersection(polygon.bounds))]
    for index, row in possible_matches_roads.iterrows():
        if polygon.intersects(row['geometry']) and row['highway'] in ['primary', 'secondary', 'tertiary', 'residential']:
            relevant_edges.append(row)

    possible_matches_lists = filtered_roads_lists.iloc[list(filtered_roads_lists_sindex.intersection(polygon.bounds))]
    for index, row in possible_matches_lists.iterrows():
        if polygon.intersects(row['geometry']):
            list_highway = row['highway']
            if any(x in ['primary', 'secondary', 'tertiary', 'residential'] for x in list_highway):
                relevant_edges.append(row)
    return relevant_edges

def place_stops_along_edges(edges, polygon, num_stops, node_size):
    # Place stops randomly along the edges within the polygon
    
    if len(edges) > 0:
        for _ in range(num_stops):
            edge = random.choice(edges)
            # Calculate the intersection between the edge and the polygon
            intersecting_line = edge['geometry'].intersection(polygon)
            if intersecting_line.is_empty:
                continue

            # Calculate the length of the intersecting part of the edge
            intersecting_length = intersecting_line.length

            # Generate a random position along the intersecting part of the edge
            random_position = random.uniform(0, intersecting_length)

            # Calculate the coordinate along the edge at the random position
            stop_location = calculate_coordinate_along_edge(intersecting_line, random_position)
            #print("Stop placed at:", stop_location)
            
            nearest_node = ox.distance.nearest_nodes(CITY_GRAPH, stop_location[0], stop_location[1])
            
            # If there is an existing node in the main graph, then add it to the list and the stop graph
            if nearest_node is not None:
                lon = CITY_GRAPH.nodes[nearest_node]['x']
                lat = CITY_GRAPH.nodes[nearest_node]['y']
                isTranspo = False
                graph_of_stops.add_node(nearest_node, lon=lon, lat=lat, isTranspo=isTranspo)
                list_of_stops.append(stopCandidate(CITY_GRAPH.nodes[nearest_node]['y'], CITY_GRAPH.nodes[nearest_node]['x'], False,  nearest_node, node_size))
        
            

def calculate_coordinate_along_edge(edge, position):
    # Calculate the coordinate along the edge at the given position
    point = edge.interpolate(position)
    return point.x, point.y
    

In [28]:
# Extract the graph into a geojson for loading into QGIS

def graph_to_geojson(graph, filename):
    # Initialize an empty list to hold GeoJSON features
    features = []

    # Iterate over the nodes in the graph
    for node, data in graph.nodes(data=True):
        # Check if the node has a geometry attribute
        if 'geometry' in data:
            # Convert the geometry to a GeoJSON-compatible format
            geometry = shapely.geometry.shape(data['geometry'])
            # Create a copy of the properties to check for NaN values
            properties = data.copy()
            # Remove the geometry from the properties
            properties.pop('geometry', None)
            # Check for NaN values in the properties
            if all(not (isinstance(value, float) and np.isnan(value)) for value in properties.values()):
                # Create a GeoJSON feature for the node
                feature = geojson.Feature(geometry=geometry, properties=properties)
                # Add the feature to the list
                features.append(feature)

    # Create a GeoJSON FeatureCollection
    feature_collection = geojson.FeatureCollection(features)

    # Return the GeoJSON FeatureCollection
    return feature_collection

## Route Network Generation

In [297]:
# V2 - Graph with the snapping function
# Generate Route Network from connected routes

# Global Variables used:
# graph_of_stops - Graph of stops that will be used to create routes
def generate_route_network(stop_nodes, max_walking_dist, connection_type="Default"):
    global route_count, used_stops
    location_road_nodes = [node for node, data in CITY_GRAPH.nodes(data=True)]
    overall_graph = nx.Graph()
    
    stop_node_coordinates = [[n.lat, n.long] for n in stop_nodes]
    stop_nodes_kd_tree = KDTree(stop_node_coordinates)
    next_nodes = [n for n in stop_nodes]
    enable_stop_nodes(next_nodes)
    route_network = []

    while not all_nodes_disabled(next_nodes) and len(next_nodes) != 0:
        selected_node = random.choice(next_nodes) # For the first node
        next_nodes.remove(selected_node)
        used_stops.append(selected_node)
        route_gen = generate_route(selected_node, next_nodes, stop_nodes_kd_tree, max_walking_dist, connection_type)
        
        if len(route_gen) > 1:
            # A route is a list of connections between two nodes
            snap_route_to_road(route_gen, overall_graph)
            route_count += 1
            
            #snapped_edges = list(snapped_route.edges(data='road_path', default=1))
            #snapped_route = connect_snapped_edges(snapped_edges)
            route_network.append(route_gen)
        else: # remove it as a route
            for con in route_gen:
                if con[0] in used_stops:
                    used_stops.remove(con[0])
                if con[-1] in used_stops:
                    used_stops.remove(con[-1])

    return route_network, overall_graph

def snap_route_to_road(route, overall_graph):
    global connection_count, graph_of_stops
    
    # Directly add nodes based on node identifiers
    for connection in route:
        overall_graph.add_node(connection[0], **graph_of_stops.nodes[connection[0]]) # The origin
        overall_graph.add_node(connection[-1], **graph_of_stops.nodes[connection[-1]]) # The destination
        
        name = f"{connection[0]}_{connection[-1]}" # "node1_node2" as name
        
        distance_travelled = 0
        # Get the total distance from point A to point B
        for i in range(len(connection)-1):
            node_data = CITY_GRAPH.nodes[connection[i]]
            next_node_data = CITY_GRAPH.nodes[connection[i+1]]
            distance_travelled += haversine(node_data['y'], node_data['x'], next_node_data['y'], next_node_data['x'])
        
        overall_graph.add_edge(connection[0], connection[-1], road_path=connection, edge_name=name, edge_id = connection_count, route_id = route_count, distance = distance_travelled) # Add edge
        connection_count += 1 # Global variable - increment the 
        
        
        if not overall_graph.has_node(connection[0]):
                print("missing ", connection[0], " in route")
        
        if not overall_graph.has_node(connection[-1]):
                print("missing ", connection[-1], " in route ")


# Generate route from stop nodes
def generate_route(source, next_nodes, stop_nodes_kd_tree, max_walking_dist, connection_type):
    short_route_list = [] # List of nx.shortest_path results
    totalDistance = 0
    selected_node = source

    while not all_nodes_disabled(next_nodes) and totalDistance < MAX_DISTANCE:
        
        #print(f"Selected node is {selected_node.getLat()}, {selected_node.getLong()}")
        disable_surrounding_nodes(next_nodes, stop_nodes_kd_tree, selected_node, max_walking_dist)
        enabled_nodes = [n for n in next_nodes if n.enabled]
        orig_node = selected_node.id
        selected_node = get_enabled_node_with_highest_edge_probability(selected_node, enabled_nodes, connection_type)
        
        if (selected_node == None or selected_node.id == orig_node):
            break
        
        next_nodes.remove(selected_node)
        dest_node = selected_node.id
        
        if not nx.has_path(CITY_GRAPH, orig_node, dest_node):
            print("No valid path found between the start and end nodes.")
        else:
            shortest_route = nx.shortest_path(CITY_GRAPH, orig_node, dest_node)
            distance_travelled = 0
            # Get the total distance from point A to point B
            for i in range(len(shortest_route)-1):
                node_data = CITY_GRAPH.nodes[shortest_route[i]]
                next_node_data = CITY_GRAPH.nodes[shortest_route[i+1]]
                
                distance_travelled += haversine(node_data['y'], node_data['x'], next_node_data['y'], next_node_data['x'])

            # Checks if it does not exceed the max distance
            if totalDistance + distance_travelled <= MAX_DISTANCE:
                
                # Updating local degree count used for connection probability
                selected_node.degree += 1
                
                totalDistance += distance_travelled
                used_stops.append(selected_node)
                short_route_list.append(shortest_route)
            else:
                break
            
    return short_route_list

# Disable surrounding nodes
def disable_surrounding_nodes(next_nodes, stop_nodes_kd_tree, source_node, max_distance):
    source = (source_node.getLat(), source_node.getLong())
    
    for node in next_nodes:
        point = (node.getLat(), node.getLong())
        distance = geodesic(source, point).meters
        if distance <= max_distance:
            node.disable()
        
def get_enabled_node_with_highest_edge_probability(source_node, enabled_nodes, connection_type):
    highest_edge_prob = 0
    highest_edge_prob_node = None

    for n in enabled_nodes:
        edge_prob = get_edge_probability(source_node, n, len(enabled_nodes), connection_type)
        if edge_prob > highest_edge_prob:
            highest_edge_prob = edge_prob
            highest_edge_prob_node = n

    return highest_edge_prob_node

# Probabilities of candidate nodes based on distance, area, node degree, and if transpo stop
def get_edge_probability(source, destination, normalization_factor, connection_type):
    source_coord = [source.getLat(), source.getLong()]
    dest_coord = [destination.getLat(), destination.getLong()]

    base_prob = exp(-(euclidean(source_coord, dest_coord))) / float(normalization_factor)

    if connection_type == "Default":
        if destination.isTranspo:
            return base_prob * 2
        return base_prob
    elif connection_type == "Area":
        if destination.isTranspo:
            return base_prob * 2
        # need to fine tune based on values of area
        # getting the distribution of all areas would be computationally costly
        return base_prob * (1 + (log(destination.getArea()) / destination.getArea()))
    elif connection_type == "Degree":
        if destination.isTranspo:
            return base_prob * 2 * (1 + (destination.getDegree() / 10))
        return base_prob * (1 + (destination.getDegree() / 10))

def radius(stops):
    circles = []
    for stop in stops:
        stop_point = Point(stop[1], stop[0])  # Create a Point object from [lat, lon] coordinates
        circle = stop_point.buffer(radius / 111000)  # Buffer the Point to create a circle (assuming 1 degree is approximately 111000 meters)
        circles.append(circle)
    return circles

def enable_stop_nodes(stop_nodes):
    for n in stop_nodes:
        n.enable()

def all_nodes_disabled(stop_nodes):
    return get_num_disabled(stop_nodes) == len(stop_nodes)

def get_num_disabled(stop_nodes):
    return sum(1 for n in stop_nodes if not n.enabled)

def haversine(lat1, lon1, lat2, lon2):
    # Use geopy's geodesic function to calculate the distance
    distance = geodesic((lat1, lon1), (lat2, lon2)).kilometers
    return distance

# Markers for visualization purposes
def add_markers(used_stops):
    for stop in used_stops:
        #popup_text = f"Name: {stop.name}<br>Type: {stop.a_type}<br>Coordinates: {stop.getLat()}, {stop.getLong()}"
        folium.Marker(location=[stop.lat, stop.long]).add_to(m)
        
        
# --------------------------------------------------------------

## Genetic Algorithm

In [211]:
# FOR TESTING ONLY
def check_routes_graph_diff():
    global list_of_networks

    num_networks = 0
    for n_network in list_of_networks:
        n_routes = n_network.routes
        n_graph = n_network.graph
        
        n_count = 0
        for n_route in n_routes:
            n_conn = 0
            for n_connection in n_route:
                if not n_graph.has_node(n_connection[0]):
                    print("missing ", n_connection[0], " in route", n_count, " in conn", n_conn, " out of ", len(n_route), flush=True)
                    break
            
                if not n_graph.has_node(n_connection[-1]):
                    print("missing ", n_connection[-1], " in route ", n_count, " in conn", n_conn, " out of ", len(n_route), flush=True)
                    break
                    
                n_conn += 1
            n_count += 1
        num_networks += 1
            

In [301]:
test_list_routes_obj = list_of_networks[0].routes
for ro in test_list_routes_obj:
    if len(ro) <= 2:
            print("WE HAVE INVALID ROUTE of LENGTH ", len(ro))
    for co in ro:
        if len(ro) <= 2:
            break
        else:
            print(co[0], " - ", co[-1])

33957031  -  34011583
34011583  -  34011580
34011580  -  3826953700
3826953700  -  243425851
243425851  -  21717801
21717801  -  25526940
25526940  -  6621936982
6621936982  -  25353083
25353083  -  32548282
32548282  -  5074966724
5074966724  -  1302744521
1302744521  -  6124583346
6124583346  -  26032767
26032767  -  1020278495
1020278495  -  5828470810
5828470810  -  3204305539
3204305539  -  9785879675
9785879675  -  26498406
26498406  -  33957110
33957110  -  33957036
33957036  -  9923680744
9923680744  -  7838739155
7838739155  -  9791018359
9791018359  -  5358463950
1732605998  -  67772940
67772940  -  7477909101
7477909101  -  161725257
161725257  -  163694861
163694861  -  1152395994
1152395994  -  990043717
990043717  -  3598103868
3598103868  -  163696786
163696786  -  9805367241
9805367241  -  6642567706
6642567706  -  29663305
29663305  -  68076077
9371332306  -  73009730
73009730  -  68065259
68065259  -  247006350
247006350  -  79637711
79637711  -  60724650
60724650  - 

In [304]:
# FOR TESTING ONLY

test_graph_net = list_of_networks[0].graph.copy()
test_routes_net = list_of_networks[0].routes.copy()
test_stops_net = list_of_networks[0].stops.copy()
test_network_obj = networkObj(test_routes_net, test_stops_net, test_graph_net)

for _ in range(5):
    mutate(test_network_obj)



PICKED NODE TO SWAP -  1904012208
OLD DISTANCE IS  14.975106677962586
MUTATION: ADDED NEW NODE TO GRAPH  159490464
MUTATION: CONNECTED NEW STOP TO EDGES
CONNECTED NEW STOP  159490464  TO PREV NODES 1202535957 ,  225937423
NEW EDGES  1202535957_159490464   159490464_225937423
MUTATION: REMOVED OLD CONNECTIONS TO NODE
MUTATION: REMOVED OLD NODE  1904012208

TESTING DIFFERENCES BETWEEN ORIGINAL AND OLD GRAPH
Unique nodes in original (not in new):
1904012208

Unique edges in original (not in new):
(1202535957, 1904012208)
(1904012208, 225937423)

Unique nodes in new (not in original):
159490464

Unique edges in new (not in original):
(225937423, 159490464)
(1202535957, 159490464)

ORIGINAL:  1904012208  -  225937423
ORIGINAL:  1202535957  -  1904012208
MODIFIED:  159490464  -  225937423
MODIFIED:  1202535957  -  159490464
MUTATION: MODIFIED THE NETWORK OBJECT'S ROUTE
Connection stop index:  0
New:  1202535957  -  159490464
Original:  1202535957  -  1904012208
New:  159490464  -  225937423


In [303]:
# This implementation takes only 2 parents from the whole generation and generates the population from them
# Instead of the what's in the paper that says the whole population will go through crossovers and mutations
# Cite Nayeem et al for GA with elitism and growing population size
def perform_genetic_algorithm(network_population, population_size, num_elites, num_generations, mutation_probability, 
                              num_mutations_probabilities, num_crossovers_probabilities, mutation_threshold_dist, num_failure_removal,
                                      weight_random_failure, weight_targeted_failure, weight_connectivity,
                              with_elitism=False, with_growing_population=False, num_mutations_per_generation=1):
    
    
    # Do this for the assigned number of generations for the GA
    for i in range(num_generations):

        new_network_population = []

        # Evaluate the fitness of each network in the population
        for network in network_population:
            road_snapped_network_graph = network.graph
            network.fitness_score = compute_fitness_score(road_snapped_network_graph, num_failure_removal,
                          weight_random_failure, weight_targeted_failure, weight_connectivity)
            print("Network score: ", network.fitness_score, flush=True)
        sorted_network_population = sorted(network_population, key=lambda x: x.fitness_score, reverse=True)
        
        # Most naive selection approach: get top two scoring networks as parents
        # But should be random with weighted probabilities so that elites are not always parents
        """
        parent1 = sorted_network_population[0]
        parent2 = sorted_network_population[1]
        """

        print("Choosing parents: ", flush=True)
        # Roulette Wheel Selection 
        # Chromosomes with higher fitness have a bigger "slice of the pie", but are not 
        # guaranteed to be selected as parents
        # This is to prevent premature convergence and ensure that the best networks are not always selected as parents
        
        fitness_scores = [network.fitness_score for network in sorted_network_population]
        min_score = min(fitness_scores)
        if min_score < 0: # Shift the scores to ensure all are positive
            fitness_scores = [score - min_score for score in fitness_scores]
        max = sum(fitness_scores)
        selection_p = [score / max for score in fitness_scores]
        
        parent1 = np.random.choice(sorted_network_population, 1, p=selection_p)[0]
        print("parent1 score, ", parent1.fitness_score, flush=True)
        sorted_network_population.remove(parent1)
        
        fitness_scores = [network.fitness_score for network in sorted_network_population]
        min_score = min(fitness_scores)
        if min_score < 0: # Shift the scores to ensure all are positive
            fitness_scores = [score - min_score for score in fitness_scores]
        max = sum(fitness_scores)
        selection_p = [score / max for score in fitness_scores]
        
        parent2 = np.random.choice(sorted_network_population, 1, p=selection_p)[0]
        print("parent2 score, ", parent2.fitness_score, flush=True)
        sorted_network_population.append(parent1)
        sorted_network_population = sorted(network_population, key=lambda x: x.fitness_score, reverse=True)


        # Take num_elites number of the best networks and automatically add them to the next generation
        if (with_elitism):
            for i in range(num_elites):
                new_network_population.append(sorted_network_population[i])

        # Ex: population_size = 20 and num_elites = 2
        # If no elitism and no growing population, then we will have 10 iterations to produce 20 in the next generation
        # Also, if elitism and growing population, then we will have 10 iterations to produce 22 in the next generation
        if (not with_elitism and not with_growing_population or with_elitism and with_growing_population):
            num_iterations = int(population_size / 2)

        # If with elitism only, maintain the population size and account for the already added elites
        elif (with_elitism):  
            num_iterations = int((population_size - num_elites) / 2)
            
        

        print("GETTING CHILDREN", flush=True)
        # Generate the population
        for i in range(num_iterations):
            
            # Get 2 children from crossovers between the two parents
            child1, child2 = crossover_split_index(parent1, parent2)
            #child1, child2 = crossover_swap_routes(parent1, parent2, num_crossovers_probabilities)
            print("FINISHED CROSSOVER")
            
            index_array = list(range(len(num_mutations_probabilities)))
            num_mutations = np.random.choice(index_array, 1, p=num_mutations_probabilities)[0]

            for j in range(num_mutations):
                # Apply mutations to the children based on mutation probability hyperparameter
                if np.random.rand() < mutation_probability:
                    mutate(child1)
                    print("MUTATION 1 DONE", flush=True)
                    
                if np.random.rand() < mutation_probability:
                    mutate(child2)
                    print("MUTATION 2 DONE", flush=True)
            
            # Add the children to the new population
            new_network_population.append(child1)
            new_network_population.append(child2)
            print("ADDED CHILDREN", flush=True)
        # Assign to next generation
        network_population = new_network_population

    return network_population

# This crossover implementation splits both networks at an index and exchanges halves
# Assumes that ideally both networks have the same number of routes (same length)
def crossover_split_index(network1, network2):
    # Split both networks at a random index
    # network.routes is a list of routes or list of lists of shortest_path
    if len(network1.routes) < len(network2.routes):
        split_index = random.randint(0, len(network2.routes)-1)
    else:
        split_index = random.randint(0, len(network1.routes)-1)
        
    # Create new graphs for the left and right sides
    route_graph1 = nx.Graph()
    route_graph2 = nx.Graph()
    
    route_network1 = []
    route_network2 = []
    
    used_stops1 = []
    used_stops2 = []
    
    count = 0
    for route in network1.routes:
        if count < split_index: # if 0-split_index -> child1 graph
            for connection in route:
                route_graph1.add_node(connection[0], **network1.graph.nodes[connection[0]])
                used_stops1.append(connection[0])
                    
                route_graph1.add_node(connection[-1], **network1.graph.nodes[connection[-1]])
                used_stops1.append(connection[-1])
                
                route_graph1.add_edge(connection[0], connection[-1], **network1.graph.get_edge_data(connection[0], connection[-1]))
            route_network1.append(route.copy())
            
                
        else: # else its for child2 graph
            for connection in route:
                route_graph2.add_node(connection[0], **network1.graph.nodes[connection[0]])
                used_stops2.append(connection[0])
                    
                route_graph2.add_node(connection[-1], **network1.graph.nodes[connection[-1]])
                used_stops2.append(connection[-1])
                
                route_graph2.add_edge(connection[0], connection[-1], **network1.graph.get_edge_data(connection[0], connection[-1]))
            route_network2.append(route.copy())
        count += 1
        
    count = 0
    for route in network2.routes:
        if count >= split_index: # if split_index-end -> child1 graph
            for connection in route:
                route_graph1.add_node(connection[0], **network2.graph.nodes[connection[0]])
                used_stops1.append(connection[0])
                    
                route_graph1.add_node(connection[-1], **network2.graph.nodes[connection[-1]])
                used_stops1.append(connection[-1])
                
                route_graph1.add_edge(connection[0], connection[-1], **network2.graph.get_edge_data(connection[0], connection[-1]))
            route_network1.append(route.copy())
                
                
        else: # else its for child2 graph
            for connection in route:
                route_graph2.add_node(connection[0], **network2.graph.nodes[connection[0]])
                used_stops2.append(connection[0])
                    
                route_graph2.add_node(connection[-1], **network2.graph.nodes[connection[-1]])
                used_stops2.append(connection[-1])
                
                route_graph2.add_edge(connection[0], connection[-1], **network2.graph.get_edge_data(connection[0], connection[-1]))
            route_network2.append(route.copy())
        count += 1
    
    child1 = networkObj(route_network1, used_stops1, route_graph1)
    child2 = networkObj(route_network2, used_stops2, route_graph2)

    return child1, child2


# Modify the stop connections of a random route in the network
# Randomly select a route and randomly select a stop in that route
# Then randomly select another stop that is a not too far from the selected stop based on threshold
# Swap connections with that stop
def mutate(network_to_mutate):
    global set_walk_distance, MAX_DISTANCE
    
    # This is to copy the original network to be compared later
    test_graph_net2 = network_to_mutate.graph.copy()
    test_routes_net2 = [copy.deepcopy(r) for r in network_to_mutate.routes]
    test_stops_net2 = network_to_mutate.stops.copy()
    copy_test_network = networkObj(test_routes_net2, test_stops_net2, test_graph_net2)
    
    # Randomly select a route
    random_route = random.choice(network_to_mutate.routes)

    
    # TODO: This is a temporary solution, it chooses either of the connections within the route except for the first and last connection
    # Randomly select a stop in the route
    # random_node_connection = random.choice(random_route) # Choose a random connection in the route
    index_array = list(range(1, len(random_route)))
    connection_index = random.choice(index_array)
    random_node_connection = random_route[connection_index]
    
    connection_stop_index = random.choice([0, -1]) # Choose whether the origin or destination node
    random_stop = random_node_connection[connection_stop_index] # The random stop to be swapped
    
    print("PICKED NODE TO SWAP - ", random_stop)
    
    # Get the old total distance
    old_total_distance = 0
    for connection in random_route:
        edge_data = network_to_mutate.graph.get_edge_data(connection[0], connection[-1])
        old_total_distance += edge_data['distance']
        
    print("OLD DISTANCE IS ", old_total_distance)
    
    
    # This is to get the subset distance (Distance without the connection with chosen random stop)
    if connection_stop_index == 0: # If it is the origin node in that connection (A, B, C and B is the chosen. Get B-C and A-B)
        prev_connection = random_route[connection_index-1] # Get the previous connection
        prev_node = prev_connection[0] # Get the origin node for that connection
        distance1 = network_to_mutate.graph.get_edge_data(prev_node, random_stop)['distance'] # Get the distance of the previous connection
        distance2 = network_to_mutate.graph.get_edge_data(random_stop, random_node_connection[-1])['distance'] # Get the distance of the current connection
        distance_to_subtract = distance1 + distance2
        
        node1 = prev_node # Setting the partner nodes
        node2 = random_node_connection[-1]
        
    else: #If it is the dest node in that connection
        distance1 = network_to_mutate.graph.get_edge_data(random_node_connection[0], random_stop)['distance'] # Get the distance of the current connection
        next_connection = random_route[connection_index+1] # Get the next route
        next_node = next_connection[-1] # Get the destination node for the next connection
        distance2 = network_to_mutate.graph.get_edge_data(random_stop, next_node)['distance']
        distance_to_subtract = distance1 + distance2
        
        node2 = next_node
        node1 = random_node_connection[0]
        
    subset_distance = old_total_distance - distance_to_subtract
    # Get the route id
    random_route_id = network_to_mutate.graph.get_edge_data(random_node_connection[0], random_node_connection[-1])['route_id']
    
    # Will try searching for a random stop 50 times (arbitrary)
    for i in range(50):
        # Get a random stop
        new_random_stop, new_random_stop_data = random.choice(list(graph_of_stops.nodes(data=True)))
        
        # Check if the new random stop is not within walking distance with the other stop in the route
        # Walking distance between node1 in route and stop to be swapped with
        source = (new_random_stop_data['lat'], new_random_stop_data['lon'])
        point = (graph_of_stops.nodes[node1]['lat'], graph_of_stops.nodes[node1]['lon'])
        walking_distance1 = geodesic(source, point).meters
        
        # Walking distance between node2 in route and stop to be swapped with
        source = (new_random_stop_data['lat'], new_random_stop_data['lon'])
        point = (graph_of_stops.nodes[node2]['lat'], graph_of_stops.nodes[node2]['lon'])
        walking_distance2 = geodesic(source, point).meters
        
        # Check if it already has been used in the route
        isCandidate = True
        for connection in random_route:
            if new_random_stop == connection[0] or new_random_stop == connection[-1]:
                isCandidate = False
                break
        
        # If it is not within walking distances, has not been used in the route, and has a path
        if walking_distance1 >= set_walk_distance and walking_distance2 >= set_walk_distance and isCandidate and nx.has_path(CITY_GRAPH, new_random_stop, node1) and nx.has_path(CITY_GRAPH, new_random_stop, node2): 
            if connection_stop_index == 0:
                shortest_route1 = nx.shortest_path(CITY_GRAPH, node1, new_random_stop)
            else:
                shortest_route1 = nx.shortest_path(CITY_GRAPH, new_random_stop, node1)
            
            distance_travelled1 = 0
            # Get the total distance from point A to point B
            for i in range(len(shortest_route1)-1):
                node_data = CITY_GRAPH.nodes[shortest_route1[i]]
                next_node_data = CITY_GRAPH.nodes[shortest_route1[i+1]]
                
                distance_travelled1 += haversine(node_data['y'], node_data['x'], next_node_data['y'], next_node_data['x'])
            
            if connection_stop_index == 0:
                shortest_route2 = nx.shortest_path(CITY_GRAPH, new_random_stop, node2)
            shortest_route2 = nx.shortest_path(CITY_GRAPH, new_random_stop, node2)
            
            distance_travelled2 = 0
            # Get the total distance from point A to point B
            for i in range(len(shortest_route2)-1):
                node_data = CITY_GRAPH.nodes[shortest_route2[i]]
                next_node_data = CITY_GRAPH.nodes[shortest_route2[i+1]]
                
                distance_travelled2 += haversine(node_data['y'], node_data['x'], next_node_data['y'], next_node_data['x'])
                
            
            # If its within the 15km distance, then this new stop can be used
            if subset_distance + distance_travelled1 + distance_travelled2 <= MAX_DISTANCE:
                # TODO: Warning it does not consider whether another route uses the old stop also before deleting
                # To be safe, we will not delete the old stop from the list
                # Modify the used stops
                network_to_mutate.stops.append(new_random_stop)
                # network.stops.remove(random_stop)
                
                # Modify the graph by adding the new node
                network_to_mutate.graph.add_node(new_random_stop, **graph_of_stops.nodes[new_random_stop])
                print("MUTATION: ADDED NEW NODE TO GRAPH ", new_random_stop)
                
                
                # Connect the new stop to the edges
                name1 = f"{shortest_route1[0]}_{shortest_route1[-1]}"
                name2 = f"{shortest_route2[0]}_{shortest_route2[-1]}"
                connection_count1 = network_to_mutate.graph.get_edge_data(node1, random_stop)['edge_id']
                connection_count2 = network_to_mutate.graph.get_edge_data(node2, random_stop)['edge_id']
                network_to_mutate.graph.add_edge(shortest_route1[0], shortest_route1[-1], road_path=shortest_route1, edge_name=name1, edge_id = connection_count1, route_id = random_route_id, distance = distance_travelled1) # Add edge
                network_to_mutate.graph.add_edge(shortest_route2[0], shortest_route2[-1], road_path=shortest_route2, edge_name=name2, edge_id = connection_count2, route_id = random_route_id, distance = distance_travelled2) # Add edge 
                
                #TODO: DELETE PRINT FOR TESTING
                print("MUTATION: CONNECTED NEW STOP TO EDGES")
                print("CONNECTED NEW STOP ", new_random_stop, " TO PREV NODES", node1, ", ", node2)
                print("NEW EDGES ", name1, " ", name2)
                
                
                
                # Remove the old connections and node if it has no other connections
                network_to_mutate.graph.remove_edge(node1, random_stop)
                network_to_mutate.graph.remove_edge(node2, random_stop)
                print("MUTATION: REMOVED OLD CONNECTIONS TO NODE")
                if (network_to_mutate.graph.degree(random_stop) == 0):
                    network_to_mutate.graph.remove_node(random_stop)
                    print("MUTATION: REMOVED OLD NODE ", random_stop)
                else:
                    print("MUTATION: DIDNT REMOVED OLD NODE ", random_stop)
                    
                
                
                # TODO: Delete FOR TESTING ONLY
                unique_nodes_G1 = set(copy_test_network.graph.nodes) - set(network_to_mutate.graph.nodes)
                unique_edges_G1 = set(copy_test_network.graph.edges) - set(network_to_mutate.graph.edges)

                unique_nodes_G2 = set(network_to_mutate.graph.nodes) - set(copy_test_network.graph.nodes)
                unique_edges_G2 = set(network_to_mutate.graph.edges) - set(copy_test_network.graph.edges)
                print()
                print("TESTING DIFFERENCES BETWEEN ORIGINAL AND OLD GRAPH")
                
                # Display unique nodes and edges for G1
                print("Unique nodes in original (not in new):")
                for node in unique_nodes_G1:
                    print(node)

                print("\nUnique edges in original (not in new):")
                for edge in unique_edges_G1:
                    print(edge)

                # Display unique nodes and edges for G2
                print("\nUnique nodes in new (not in original):")
                for node in unique_nodes_G2:
                    print(node)

                print("\nUnique edges in new (not in original):")
                for edge in unique_edges_G2:
                    print(edge)
                    
                print()
                
                # Modify the route
                if connection_stop_index == 0: #If its the origin node
                    random_route[connection_index] = shortest_route2 # Change the current connection
                    random_route[connection_index-1] = shortest_route1 # Change the previous connection
                    
                    #TODO: DELETE FOR TESTING
                    random_route2 = copy_test_network.routes[network_to_mutate.routes.index(random_route)]
                    print("ORIGINAL: ", random_route2[connection_index][0], " - ", random_route2[connection_index][-1])
                    print("ORIGINAL: ", random_route2[connection_index-1][0], " - ", random_route2[connection_index-1][-1])
                    print("MODIFIED: ", random_route[connection_index][0], " - ", random_route[connection_index][-1])
                    print("MODIFIED: ", random_route[connection_index-1][0], " - ", random_route[connection_index-1][-1])
                    
                else: #else If its the dest node
                    random_route[connection_index] = shortest_route1 # Change the current connection
                    random_route[connection_index+1] = shortest_route2 # Change the next connection
                    
                    #TODO: DELETE FOR TESTING
                    random_route2 = copy_test_network.routes[network_to_mutate.routes.index(random_route)]
                    print("ORIGINAL: ", random_route2[connection_index][0], " - ", random_route2[connection_index][-1])
                    print("ORIGINAL: ", random_route2[connection_index+1][0], " - ", random_route2[connection_index+1][-1])
                    print("MODIFIED: ", random_route[connection_index][0], " - ", random_route[connection_index][-1])
                    print("MODIFIED: ", random_route[connection_index+1][0], " - ", random_route[connection_index+1][-1])
                    
                print("MUTATION: MODIFIED THE NETWORK OBJECT'S ROUTE")
                
                # TODO: Delete FOR TESTING - Displaying each route
                
                print("Connection stop index: ", connection_stop_index)
                for r in range(len(network_to_mutate.routes)):
                    for c in range(len(network_to_mutate.routes[r])):
                        if network_to_mutate.routes[r][c][0] != copy_test_network.routes[r][c][0] or network_to_mutate.routes[r][c][-1] != copy_test_network.routes[r][c][-1]:
                            print("New: ", network_to_mutate.routes[r][c][0], " - ", network_to_mutate.routes[r][c][-1])
                            print("Original: ", copy_test_network.routes[r][c][0], " - ", copy_test_network.routes[r][c][-1])
                        
                
                # Break the loop once we swap
                break  
                


### Fitness Function

In [210]:
# Fitness function

def select_highest_scoring_mutation(candidate_road_snapped_networks, num_failure_removal,
                                    weight_random_failure, weight_targeted_failure, weight_radius_of_gyration):
    max_fitness_score = -np.inf
    max_candidate_route_snapped_network = None

    for n in candidate_road_snapped_networks:
        fitness_score = compute_fitness_score(n, num_failure_removal,
                                              weight_random_failure, weight_targeted_failure, weight_radius_of_gyration)
        if fitness_score > max_fitness_score:
            max_fitness_score = fitness_score
            max_candidate_route_snapped_network = n

    return max_candidate_route_snapped_network

def compute_fitness_score(road_snapped_network_graph, num_failure_removal,
                          weight_random_failure, weight_targeted_failure, weight_radius_of_gyration):

    random_failure_robustness = compute_random_failure_robustness(road_snapped_network_graph, num_failure_removal)
    weighted_random_failure_robustness = weight_random_failure * random_failure_robustness

    targeted_failure_robustness = compute_targeted_failure_robustness(road_snapped_network_graph, num_failure_removal)
    weighted_targeted_failure_robustness = weight_targeted_failure * targeted_failure_robustness

    radius_of_gyration = compute_connectivity(road_snapped_network_graph)
    weighted_radius_of_gyration = weight_radius_of_gyration * radius_of_gyration
    
    #print("Random Failure Score: ", weighted_random_failure_robustness)
    #print("Target Failure Score: ", weighted_targeted_failure_robustness)
    #print("Connectivity: ", weighted_radius_of_gyration)

    # Will use this return for now to utilize target and random failure nodes 
    return weighted_radius_of_gyration - weighted_random_failure_robustness - weighted_targeted_failure_robustness
    # return weighted_radius_of_gyration


### WRITTEN IN PSEUDOCODE
def compute_connectivity(network):
    # External connectivity - measure how connected is the jeepney route network with other modes of transpo
    
    # Get the ratio of transportation stops to total stops in the network
    transpo_stops = [node for node, node_data in network.nodes(data=True) if node_data['isTranspo'] == True]
    total_stops = len(network.nodes(data=True))
    transpo_stop_ratio = len(transpo_stops) / total_stops

    # Get the average degree of all transportation stops in the network
    if len(transpo_stops) > 0:
        avg_transpo_degree = sum(network.degree(stop) for stop in transpo_stops) / len(transpo_stops)
    else:
        avg_transpo_degree = 1

    # Find a way to normalize the two values and combine them 

    # Internal connectivity - measure how connected is each jeepney route to other jeepney routes
                    
    # This counts how many nodes have intersections (Meaning node is connected to more than one route by route ID)
    num_intersections = 0
    for node, node_data in network.nodes(data=True):
        connected_edges = network.edges(node)
        unique_route_id = []
        
        for edge in connected_edges:
            route_id = network[edge[0]][edge[1]]['route_id']
            
            if route_id not in unique_route_id:
                unique_route_id.append(route_id)
                
        if len(unique_route_id) > 1:
            num_intersections += 1

    
    # Change these weights based on what the expected values for 
    # the transpo_stop_ratio, avg_transpo_degree, and num_intersections will be
    external_weight = 0.5
    internal_weight = 0.5
    
    # TODO: Delete this
    #print("Transpo stop ratio: ", transpo_stop_ratio)
    #print("Num intersections: ", num_intersections)
    #print("Average degree: ", avg_transpo_degree)

    # Formula subject to change
    return external_weight * (transpo_stop_ratio * avg_transpo_degree) + internal_weight * num_intersections


def compute_random_failure_robustness(road_snapped_network_graph, num_removals):
    graph_copy = road_snapped_network_graph.copy() # Make a copy
    
    for i in range(num_removals):
        selected_node = random.choice(list(graph_copy.nodes()))
        graph_copy.remove_node(selected_node)

    diameter, avg_path_length = compute_network_statistics(graph_copy)
    return compute_failure_robustness(graph_copy, diameter)

def compute_targeted_failure_robustness(road_snapped_network_graph, num_removals):
    graph_copy = road_snapped_network_graph.copy() # Make a copy
    
    for i in range(num_removals):
        node_degrees = graph_copy.degree()
        # Iterate over the DegreeView object to find the maximum degree
        max_degree = max(degree for _, degree in node_degrees)
        max_degree_node = get_node_with_degree(node_degrees, max_degree)
        graph_copy.remove_node(max_degree_node)

    diameter, avg_path_length = compute_network_statistics(graph_copy)
    return compute_failure_robustness(graph_copy, diameter)

def compute_failure_robustness(road_snapped_network_graph, max_path_length):
    return float(max_path_length) / float(len(road_snapped_network_graph) - 1)

def compute_network_statistics(road_snapped_network_graph):
    path_lengths = get_path_lengths(road_snapped_network_graph)
    avg_path_length = np.mean(path_lengths)
    max_path_length = max(path_lengths)

    #network_size = len(path_lengths)
    #gcc = sorted(nx.connected_component_subgraphs(road_snapped_network_graph), key=len, reverse=True)
    #giant_component_fraction = float(float(gcc[0].order()) / float(network_size))
    #return max_path_length, avg_path_length, giant_component_fraction
    return max_path_length, avg_path_length

def get_node_with_degree(node_degrees, degree):
    # Iterate over the DegreeView object to find the node with the specified degree
    for node, _ in node_degrees:
        if _ == degree:
            return node
    return None  # Return None if no node with the specified degree is found

def get_path_lengths(snapped_road_network_graph):
    return [sum(nx.single_source_shortest_path_length(snapped_road_network_graph, n).values())
            for n in snapped_road_network_graph]

### Extra Code Just in Case

In [32]:
# def compute_radius_of_gyration(road_snapped_network_graph, num_random_values, weight):
#     return _get_efficiency_sum(road_snapped_network_graph, num_random_values, weight)

# def _get_efficiency_sum(graph, no_of_random_values, weight):
#     efficiency_sum = 0.0
#     weighted_list = _get_yweighted_list(graph, weight)
#     efficiency_sum_list = random.sample(weighted_list.keys(), no_of_random_values)

#     for k_x, k_y in efficiency_sum_list:
#          temp = weighted_list[(str(k_x), str(k_y))]
#          efficiency_sum = float(temp) + float(efficiency_sum)

#     return efficiency_sum

# def _get_yweighted_list(graph, weight):
#     dp = _get_distance_individual(graph)
#     dw = _get_total_weighted_distance(graph, weight)
#     Y = {}

#     for k_x, v_x in graph.nodes(data=True):
#         for k_y, v_y in graph.nodes(data=True):
#             if nx.has_path(CITY_GRAPH, k_x, k_y):
#                 Y[(k_x, k_y)] = float(dp[(k_x, k_y)])/float(dw)
#             else:
#                 Y[(k_x, k_y)] = 0.0

#     return Y

# def _get_distance_individual(graph):
#     T = {}

#     for k_x, v_x in graph.nodes(data=True):
#         for k_y, v_y in graph.nodes(data=True):
#             if nx.has_path(CITY_GRAPH, k_x, k_y):
#                 shortest_path_nodes = nx.shortest_path(CITY_GRAPH, k_x, k_y)
#                 accumulated_distance = 0.0
#                 if len(shortest_path_nodes) > 1:
#                     for i in range(0, len(shortest_path_nodes) - 1):
#                         edge = CITY_GRAPH.get_edge_data(shortest_path_nodes[i], shortest_path_nodes[i + 1]).get('dist', 0)
#                         print(edge)
#                         accumulated_distance += float(edge)
#                     T[(k_x, k_y)] = accumulated_distance
#                 else:
#                     T[(k_x, k_y)] = 0
#             else:
#                 T[(k_x, k_y)] = 0

#     return T

# # new gettwd does not use weighted adjacency matrix
# def _get_total_weighted_distance(graph, weight):
#     # A = _create_weighted_adjacency_matrix(graph)
#     dp = _get_distance_individual(graph)
#     w = weight
#     total_weighted_distance = 0.0
#     T = {}
#     for k_x, v_x in graph.nodes(data=True):
#         for k_y, v_y in graph.nodes(data=True):
#             if nx.has_path(CITY_GRAPH, k_x, k_y):
#                 shortest_path_nodes = nx.shortest_path(CITY_GRAPH, k_x, k_y)
#                 g = get_nodes_shortest_path(shortest_path_nodes, graph)
#                 T = _get_no_of_transfers(g)
#                 a = 1.0
#             elif not nx.has_path(CITY_GRAPH, k_x, k_y):
#                 a = 10.0

#             b = float(dp[(k_x, k_y)])
#             weighted_distance = a * b + (w * T)
#             total_weighted_distance = float(total_weighted_distance) + float(weighted_distance)

#     return total_weighted_distance



# def get_nodes_shortest_path(shortest_path_nodes):
#     new_graph = nx.Graph()

#     for k_y, v_y in CITY_GRAPH.nodes(data=True):
#         for elem in shortest_path_nodes:
#             if elem == k_y:
#                 new_graph.add_node(k_y, lat=v_y.get('lat'), lon=v_y.get('lon'), route_id = v_y.get('route_id'))

#     return new_graph


# def _get_no_of_transfers(graph):
#     temp = []
#     p = graph.copy()
#     no_of_tranfer = 0

#     for k_y, v_y in p.nodes(data=True):
#         if (len(p.nodes()) > 1):
#             if str(v_y.get("route_id")) not in temp:
#                 temp.append(str(v_y.get("route_id")))
#             no_of_transfer = len(temp)-1
#         else:
#             no_of_tranfer = 0

#     return no_of_tranfer

In [33]:
# # SNAPPING ROUTE NETWORK TO GRAPH
# # NO UUID V1

# def snap_route_network_to_road(route_network):
#     location_road_nodes = [node for node, data in graph.nodes(data=True)]
#     snapped_route_network = []
#     snapped_route_network_graph = []
    
#     overall_graph = nx.Graph()

#     route_id = 0
#     for route in route_network:
#         snapped_route = snap_route_to_road(location_road_nodes, route)
#         snapped_route_network_graph.append(snapped_route)
#         nx.set_edge_attributes(snapped_route, 'route_id', route_id)
#         route_id += 1
#         overall_graph = nx.compose(overall_graph, snapped_route)

#         snapped_edges = list(snapped_route.edges(data='road_path', default=1))
#         snapped_route = connect_snapped_edges(snapped_edges)
#         snapped_route_network.append(snapped_route)
        
#     snapped_route_graph = nx.union_all(snapped_route_network_graph)

#     return snapped_route_network, snapped_route_graph



# def connect_snapped_edges(snapped_edges):
#     connected_edge = []

#     while len(snapped_edges) > 0:
#         curr_edge = consecutive_connect([], snapped_edges.pop(0))

#         for e in snapped_edges:
#             consecutive_edge = consecutive_connect(curr_edge, e)
#             if consecutive_edge is not None:
#                 curr_edge = consecutive_edge
#                 snapped_edges.remove(e)

#         new_connected_edge = consecutive_connect(connected_edge, curr_edge)
#         connected_edge = new_connected_edge if new_connected_edge is not None else connected_edge

#     return connected_edge

# def consecutive_connect(e1, e2):
#     if len(e1) == 0:
#         return e2
#     elif len(e2) == 0:
#         return e1

#     if e1[-1] == e2[0]:
#         return e1 + e2
#     elif e2[-1] == e1[0]:
#         return e2 + e1
#     else:
#         return None

# def snap_route_to_road(location_road_nodes, route_stop_nodes):
#     snapped_route = nx.Graph()
#     route_stops = [] # We need to store the list of stops as route_stop_nodes is only a list of ox.shortest_path results
    
    
#     # Directly add nodes based on node identifiers
#     for n in route_stop_nodes:
#         snapped_route.add_node(n[0], **graph.nodes[n[0]])
#         route_stops.append(n[0])
#         last_path = n # For the sake of getting the last stop of the last stop connection
        
#     snapped_route.add_node(n[len(last_path)-1], **graph.nodes[n[len(last_path)-1]])
#     route_stops.append(n[len(last_path)-1])

#     # Add the shortest path for each consecutive node as an edge in the graph
#     for i in range(len(route_stops) - 1):
#         source_node = route_stops[i]
#         dest_node = route_stops[i + 1]
#         shortest_road_path = get_shortest_road_path(location_road_nodes, source_node, dest_node)
#         snapped_route.add_edge(source_node, dest_node, road_path=shortest_road_path)
#     return snapped_route

# def get_shortest_road_path(location_road_nodes, source_stop_node, dest_stop_node):
#     # Find the nearest nodes in the graph to the source and destination GeoPoints
#     closest_road_node_to_source = ox.distance.nearest_nodes(graph, graph.nodes[source_stop_node]['x'], graph.nodes[source_stop_node]['y'])
#     closest_road_node_to_dest = ox.distance.nearest_nodes(graph,  graph.nodes[dest_stop_node]['x'], graph.nodes[dest_stop_node]['y'])

#     if nx.has_path(graph, closest_road_node_to_source, closest_road_node_to_dest):
#         return nx.shortest_path(graph, closest_road_node_to_source, closest_road_node_to_dest)
#     else:
#         return {}

In [34]:
# # Generate Route Network from connected routes

# # Global Variables used:
# # graph_of_stops - Graph of stops that will be used to create routes
# def generate_route_network(stop_nodes, max_walking_dist):
#     stop_node_coordinates = [[n.lat, n.long] for n in stop_nodes]
#     stop_nodes_kd_tree = KDTree(stop_node_coordinates)
#     next_nodes = [n for n in stop_nodes]
#     enable_stop_nodes(next_nodes)
#     route_network = []

#     #Create empty graph
#     new_graph = graph_of_stops.copy() # Copy of Manila

    
#     while not all_nodes_disabled(next_nodes) and len(next_nodes) != 0:
#         selected_node = random.choice(next_nodes) # For the first node
#         next_nodes.remove(selected_node)
#         used_stops.append(selected_node)
        
#         route_network.append(generate_route(selected_node, next_nodes, stop_nodes_kd_tree, max_walking_dist, new_graph))

#     return route_network, new_graph

# # Generate route from stop nodes
# def generate_route(source, next_nodes, stop_nodes_kd_tree, max_walking_dist, new_graph):
#     route = []
#     totalDistance = 0
#     selected_node = source

#     while not all_nodes_disabled(next_nodes) and totalDistance < MAX_DISTANCE:
        
#         #print(f"Selected node is {selected_node.getLat()}, {selected_node.getLong()}")
#         disable_surrounding_nodes(next_nodes, stop_nodes_kd_tree, selected_node, max_walking_dist)
#         enabled_nodes = [n for n in next_nodes if n.enabled]
#         orig_node = ox.distance.nearest_nodes(new_graph, selected_node.getLong(), selected_node.getLat()) # Getting the node from the graph itself
#         old_node = selected_node
#         selected_node = get_enabled_node_with_highest_edge_probability(selected_node, enabled_nodes)
        
#         if (selected_node == None or selected_node == old_node):
#             break
        
#         next_nodes.remove(selected_node)
#         dest_node = ox.distance.nearest_nodes(new_graph, selected_node.getLong(), selected_node.getLat())
        
#         if orig_node is None or dest_node is None:
#             print("Unable to find valid nodes. Please verify the start and end coordinates.")
#         elif not nx.has_path(graph, orig_node, dest_node):
#             print("No valid path found between the start and end nodes.")
#         else:
            
#             shortest_route = nx.shortest_path(graph, orig_node, dest_node)
#             distance_travelled = 0
#             # Get the total distance from point A to point B
#             for i in range(len(shortest_route)-1):
#                 node_data = graph.nodes[shortest_route[i]]
#                 next_node_data = graph.nodes[shortest_route[i+1]]
                
#                 distance_travelled += haversine(node_data['y'], node_data['x'], next_node_data['y'], next_node_data['x'])

#             # Checks if it does not exceed the max distance
#             if totalDistance + distance_travelled <= MAX_DISTANCE:
#                 # Add the nodes to the nx graph
#                 new_graph.add_edge(orig_node, dest_node, weight=distance_travelled)
                
#                 totalDistance += distance_travelled
#                 used_stops.append(selected_node)
#                 route.append(shortest_route)
#             else:
#                 break
#     return route

# # Disable surrounding nodes
# def disable_surrounding_nodes(next_nodes, stop_nodes_kd_tree, source_node, max_distance):
#     source = (source_node.getLat(), source_node.getLong())
    
#     for node in next_nodes:
#         point = (node.getLat(), node.getLong())
#         distance = geodesic(source, point).meters
#         if distance <= max_distance:
#             node.disable()
        
# def get_enabled_node_with_highest_edge_probability(source_node, enabled_nodes):
#     highest_edge_prob = 0
#     highest_edge_prob_node = None

#     for n in enabled_nodes:
#         edge_prob = get_edge_probability(source_node, n, len(enabled_nodes))
#         if edge_prob > highest_edge_prob:
#             highest_edge_prob = edge_prob
#             highest_edge_prob_node = n

#     return highest_edge_prob_node


# def get_edge_probability(source, destination, normalization_factor):
#     source_coord = [source.getLat(), source.getLong()]
#     dest_coord = [destination.getLat(), destination.getLong()]
#     return exp(-(euclidean(source_coord, dest_coord))) / float(normalization_factor)


# def radius(stops):
#     circles = []
#     for stop in stops:
#         stop_point = Point(stop[1], stop[0])  # Create a Point object from [lat, lon] coordinates
#         circle = stop_point.buffer(radius / 111000)  # Buffer the Point to create a circle (assuming 1 degree is approximately 111000 meters)
#         circles.append(circle)
#     return circles

# def enable_stop_nodes(stop_nodes):
#     for n in stop_nodes:
#         n.enable()

# def all_nodes_disabled(stop_nodes):
#     return get_num_disabled(stop_nodes) == len(stop_nodes)

# def get_num_disabled(stop_nodes):
#     return sum(1 for n in stop_nodes if not n.enabled)



# def haversine(lat1, lon1, lat2, lon2):
#     # Use geopy's geodesic function to calculate the distance
#     distance = geodesic((lat1, lon1), (lat2, lon2)).kilometers
#     return distance

# # Markers for visualization purposes
# def add_markers(used_stops):
#     for stop in used_stops:
#         #popup_text = f"Name: {stop.name}<br>Type: {stop.a_type}<br>Coordinates: {stop.getLat()}, {stop.getLong()}"
#         folium.Marker(location=[stop.road_lat, stop.road_long]).add_to(m)

In [ ]:
# def connect_snapped_edges(snapped_edges):
#     connected_edge = []

#     while len(snapped_edges) > 0:
#         curr_edge = consecutive_connect([], snapped_edges.pop(0))

#         for e in snapped_edges:
#             consecutive_edge = consecutive_connect(curr_edge, e)
#             if consecutive_edge is not None:
#                 curr_edge = consecutive_edge
#                 snapped_edges.remove(e)

#         new_connected_edge = consecutive_connect(connected_edge, curr_edge)
#         connected_edge = new_connected_edge if new_connected_edge is not None else connected_edge

#     return connected_edge


# def consecutive_connect(e1, e2):
#     if len(e1) == 0:
#         return e2
#     elif len(e2) == 0:
#         return e1

#     if e1[-1] == e2[0]:
#         return e1 + e2
#     elif e2[-1] == e1[0]:
#         return e2 + e1
#     else:
#         return None



# This crossover implementation randomly selects a route from each network and swaps them
# More similar to the previous thesis implementation
# num_crossovers_probabilities = list of probabilities for crossovers that will be randomly selected from
#    e.g. [0.1, 0.2, 0.3, 0.4, 0.5] would mean a 0.1 probability for 0 crossovers, 0.2 probability for 1 crossover, etc
# def crossover_swap_routes(network1, network2, num_crossovers_probabilities):
#     num_crossovers = np.random.choice(len(num_crossovers_probabilities), 1, p=num_crossovers_probabilities)[0]

#     for i in range(num_crossovers):
#         # Randomly select a route from each network
#         route1 = random.choice(network1.items())
#         route2 = random.choice(network2.items())

#         # Swap the routes
#         network1[route1[0]] = route2[1]
#         network2[route2[0]] = route1[1]

#     return network1, network2

## Main

### Creating the zones

In [35]:
# CREATING INITIAL NETWORK

manila_amenities_network = create_network(manila_amenities_polygon_gdf, manila_amenities_point_gdf)

# Make a before map
before_map = plot_network_on_map(manila_amenities_network, initial_location=[0, 0], zoom_start=100)
before_map.save('before_map.html') # Save the map to an HTML file

In [36]:
# LEVEL 1 - CONNECTING POLYGONS OF SAME AMENITY

combined_graph = combine_amenities_by_polygon(manila_amenities_network, max_distance=100, max_perimeter=10000)
after_map = plot_network_on_map(combined_graph, initial_location=[0, 0], zoom_start=100)
after_map.save('after_map.html') # Save the map to an HTML file

In [37]:
# CHECKING POPULATION DENSITY OF RESIDENTIAL AREAS

pop_graph = check_residential_population_density(graph=combined_graph, threshold=100)
pop_map = plot_population_zones_map(pop_graph, initial_location=[0, 0], zoom_start=100)

In [38]:
# LEVEL 2 - CREATING NETWORKS OF AMENITIES

graph_networks_of_polygons = create_zone_network(graph=combined_graph, max_distance=100)
networks_map = plot_connected_zones_network_on_map(graph_networks_of_polygons, initial_location=[0, 0], zoom_start=100)
networks_map.save('networks_map.html') # Save the map to an HTML file

feature_collection = graph_to_geojson(manila_amenities_network, 'output.geojson')
with open('output.geojson', 'w', encoding='utf-8') as f:
    f.write(geojson.dumps(feature_collection, indent=2))

### Placing Stops

In [39]:
# LEVEL 3 - CREATING STOPS TO BE PLACED ON ZONES
graph_of_stops = nx.Graph()
add_points_to_graph(manila_amenities_network, graph_networks_of_polygons) # Add first all transportation stops
list_of_stops = []
place_stops_on_roads(graph_networks_of_polygons) # Adds stops graph_of_stops

# Visualize the stops
stops_map = plot_stops_on_map(networks_map, list_of_stops, initial_location=[0, 0], zoom_start=100)
stops_map.save('stops_map.html') # Save the map to an HTML file

In [40]:
#Export stops to pickle

# Specify the file path where you want to save the pickle file
file_path = 'stop_objects.pkl'

# Open the file in binary write mode
with open(file_path, 'wb') as f:
    # Dump the list of stop objects into the pickle file
    pickle.dump(list_of_stops, f)

In [41]:
# Import and Export networks or graphs to pickle
def export_networks(networks, path):
    with open(path, 'wb') as f:
        pickle.dump(networks, f)

def import_networks(path):
    with open(path, 'rb') as f:
        routes = pickle.load(f)
    return routes

### Connecting Stops

In [298]:
# LEVEL 4 - CONNECTING STOPS INTO A NETWORK
WALKING_DISTANCES = [300,550,800]
MAX_DISTANCE = 15
CONNECTION_TYPES = ["Default", "Area", "Degree"]

# Configuration
set_walk_distance = WALKING_DISTANCES[0]
num_of_networks = 10
conn_type = CONNECTION_TYPES[0]
        
# Generate route network
list_of_networks = []

for _ in range(num_of_networks):
    route_count = 0 # Route is the connection between list of stops
    connection_count = 0 # Connection is the connection between two stops / nodes
    used_stops = []
    route_network, route_graph = generate_route_network(list_of_stops, set_walk_distance, conn_type) # Default max walking distance is 300m
    
    # or a network object
    list_of_networks.append(networkObj(route_network, used_stops, route_graph))


#Export networks and graphs using pickl
export_networks(list_of_networks, "networks.pkl")


i = 1
# Creating Maps for visualization
for route_network in list_of_networks:
    map_center = (14.599512, 120.984222)
    m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

    # Plotting in the Map
    add_markers(route_network.stops)
        
    for route in route_network.routes:
        for connection in route:
            ox.plot_route_folium(CITY_GRAPH, connection, route_map=m, tiles='openstreetmap', route_color="green")

    m.save(f"Map2-{i}.html")
    i += 1


No valid path found between the start and end nodes.
No valid path found between the start and end nodes.
No valid path found between the start and end nodes.
No valid path found between the start and end nodes.
No valid path found between the start and end nodes.
No valid path found between the start and end nodes.
No valid path found between the start and end nodes.
No valid path found between the start and end nodes.
No valid path found between the start and end nodes.
No valid path found between the start and end nodes.
No valid path found between the start and end nodes.
No valid path found between the start and end nodes.
No valid path found between the start and end nodes.
No valid path found between the start and end nodes.
No valid path found between the start and end nodes.
No valid path found between the start and end nodes.
No valid path found between the start and end nodes.
No valid path found between the start and end nodes.
No valid path found between the start and end 

/var/folders/q0/zbq8f0v11vb5hv6jh21spfwm0000gn/T/ipykernel_16850/1088053865.py:39: FutureWarning: The `folium` module has been deprecated and will be removed in the v2.0.0 release. You can generate and explore interactive web maps of graph nodes, edges, and/or routes automatically using GeoPandas.GeoDataFrame.explore instead, for example like: `ox.graph_to_gdfs(G, nodes=False).explore()`. See the OSMnx examples gallery for complete details and demonstrations.
  ox.plot_route_folium(CITY_GRAPH, connection, route_map=m, tiles='openstreetmap', route_color="green")
/var/folders/q0/zbq8f0v11vb5hv6jh21spfwm0000gn/T/ipykernel_16850/1088053865.py:39: FutureWarning: The `folium` module has been deprecated and will be removed in the v2.0.0 release. You can generate and explore interactive web maps of graph nodes, edges, and/or routes automatically using GeoPandas.GeoDataFrame.explore instead, for example like: `ox.graph_to_gdfs(G, nodes=False).explore()`. See the OSMnx examples gallery for compl

### Genetic Algorithm

In [305]:
# Import networks into list_of_networks
# import pickl
list_of_networks = import_networks("networks.pkl")

In [306]:
# TESTING GA ---------------------------------------------------------]
# There should a pickle file already of the latest networks

population_size = 20 # Default
num_elites = 2
num_generations = 10
mutation_probability = 0.1
num_mutations_probabilities = [0.1, 0.2, 0.3, 0.2, 0.2]
num_crossovers_probabilities = [0.1, 0.2, 0.3, 0.2, 0.2]
mutation_threshold_dist = 300
with_elitism = False
with_growing_population = False
num_mutations_per_generation = 2

#Weights and Fitness Function configuration
num_failure_removal = 4
weight_random_failure = 0.6
weight_targeted_failure = 0.1
weight_connectivity = 0.4

population = perform_genetic_algorithm(list_of_networks, population_size, num_elites, num_generations, mutation_probability, 
                              num_mutations_probabilities, num_crossovers_probabilities, mutation_threshold_dist, num_failure_removal,
                                      weight_random_failure, weight_targeted_failure, weight_connectivity,
                              with_elitism, with_growing_population, num_mutations_per_generation)

Network score:  -0.35021097046413496
Network score:  -0.8524530487300405
Network score:  -0.5503980891719745
Network score:  -0.4079999999999999
Network score:  -0.7626031201502899
Network score:  -0.8899256100879976
Network score:  -0.9289516129032258
Network score:  -0.8665018597478001
Network score:  -1.047878625134264
Network score:  -0.5430817610062892
Choosing parents: 
parent1 score,  -0.5503980891719745
parent2 score,  -0.8899256100879976
GETTING CHILDREN
FINISHED CROSSOVER
PICKED NODE TO SWAP -  3077651962
OLD DISTANCE IS  10.275242664813286
MUTATION: ADDED NEW NODE TO GRAPH  68073682
MUTATION: CONNECTED NEW STOP TO EDGES
CONNECTED NEW STOP  68073682  TO PREV NODES 252166905 ,  261741171
NEW EDGES  252166905_68073682   68073682_261741171
MUTATION: REMOVED OLD CONNECTIONS TO NODE
MUTATION: REMOVED OLD NODE  3077651962

TESTING DIFFERENCES BETWEEN ORIGINAL AND OLD GRAPH
Unique nodes in original (not in new):
3077651962

Unique edges in original (not in new):
(252166905, 30776519

IndexError: list index out of range

### Testing only

In [300]:
for n_network in list_of_networks:
    n_routes = n_network.routes
    n_graph = n_network.graph
    
    count = 0
    for n_route in n_routes:
        n_conn = 0
        for n_connection in n_route:
            if not n_graph.has_node(n_connection[0]):
                print("missing ", n_connection[0], " in route", count, " in conn", n_conn, " out of ", len(n_route))
        
            if not n_graph.has_node(n_connection[-1]):
                print("missing ", n_connection[-1], " in route ", count, " in conn", n_conn, " out of ", len(n_route))
                
            n_conn += 1
        
        count += 1

In [134]:
# TEST DELETE AFTER
test_net = list_of_networks[0]
route_test = test_net.routes[0]

count = 0
for connection in route_test:
    if not test_net.graph.has_node(connection[0]):
        print("missing ", connection[0], " in ", count)
        
    if not test_net.graph.has_node(connection[-1]):
        print("missing ", connection[-1], " in ", count)
        
    count += 1

In [ ]:
# TEST DELETE AFTER
test_net = list_of_networks[0]
route_test = test_net.routes[0]

for connection in route_test:
    if test_net.graph.has_node(connection[0]):
        print("Has origin")
    else:
        print("missing origin")
        
    if test_net.graph.has_node(connection[-1]):
        print("Has dest")
    else:
        print("missing dest")

In [ ]:
# TEST DELETE AFTER

for connection in route_test:
    if not graph_of_stops.has_node(connection[0]):
        print("missing origin")
        
    if not graph_of_stops.has_node(connection[-1]):
        print("missing dest")

In [ ]:
for node, node_data in graph_of_stops.nodes(data=True):
    # Print the node identifier
    
    if node_data['isTranspo'] == True:
    #if node == 5828739366:
        # Access and print each attribute key-value pair
        print(f"Node ID: {node}")
        for attr_name, attr_value in node_data.items():
            print(f"{attr_name}: {attr_value}")

In [ ]:
for node, node_data in manila_amenities_network.nodes(data=True):
    # Print the node identifier
    
    if node_data['amenity'] == 'transportation':
    #if node == 5828739366:
        # Access and print each attribute key-value pair
        print(f"Node ID: {node}")
        for attr_name, attr_value in node_data.items():
            print(f"{attr_name}: {attr_value}")

In [ ]:
for node, node_data in list_of_networks[0].graph.nodes(data=True):
    # Print the node identifier
    print(f"Node ID: {node}")
    
    # Access and print each attribute key-value pair
    for attr_name, attr_value in node_data.items():
        print(f"{attr_name}: {attr_value}")

## NEW FILES

In [ ]:
# Create spatial index for points
idx = index.Index()
for j, point in merged_amenities_points_gdf.iterrows():
    idx.insert(j, point['geometry'].bounds)

# Iterate over polygons
for i, polygon in merged_amenities_polygons_gdf.iterrows():
    points_within_polygon = []
    
    # Iterate over points within the bounding box of the polygon
    for j in idx.intersection(polygon['geometry'].bounds):
        point = merged_amenities_points_gdf.loc[j]
        if polygon['geometry'].intersects(point['geometry']):
            points_within_polygon.append(j)
    
    merged_amenities_polygons_gdf.at[i, 'amenity_points'] = points_within_polygon

In [ ]:
manila_amenities_polygon_gdf['amenity'].unique()

In [ ]:
merged_amenities_polygons_gdf['amenity'].unique()

In [ ]:
# Creating Initial network
merged_amenities_network = create_network(merged_amenities_polygons_gdf, merged_amenities_points_gdf)

# Make a before map
merge_map = plot_network_on_map(merged_amenities_network, initial_location=[0, 0], zoom_start=100)
merge_map.save('merge_map.html') # Save the map to an HTML file

In [ ]:
# JUST TO VISUALIZE RIVERS AND STREAMS
map_center = (14.599512, 120.984222)  # TEMPORARY WILL ZOOM TO MANILA
m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

# Iterate over the nodes in the network
folium.GeoJson(filtered_rivers, style_function=lambda x: {'color': 'blue'}).add_to(m)

m.save('check.html')  # Save the map to an HTML file

In [ ]:
# Connecting polygons of same amenity
connected_lines = []
combined_graph = combine_amenities_by_polygon(manila_amenities_network, max_distance=100, max_perimeter=10000)
after_map = plot_network_on_map(combined_graph, initial_location=[0, 0], zoom_start=100)


# The lines to show the networks
for line in connected_lines:
    line_coords = [[coord[1], coord[0]] for coord in line.coords]
    folium.PolyLine(locations=line_coords, color='black').add_to(after_map)
after_map.save('after_map.html') # Save the map to an HTML file

## MISC (VISUALIZATION)

In [ ]:
# FOR VISUALIZATION ONLY
all_roads_map = plot_all_roads()
all_roads_map.save('all_roads.html')

filtered_road_map = plot_all_filtered_roads()
filtered_road_map.save('filtered_road_map.html')

In [ ]:
merged_amenities_points_gdf['amenity'].unique()

In [ ]:
map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

for j, point in merged_amenities_points_gdf.iterrows():
    if point['amenity'] == 'grocery':
        folium.Marker(location=[point['y'], point['x']], popup=f"{point['name']}").add_to(m)
        
m.save('test.html')

In [ ]:
def plot_amenity_test(amenities_network, amenity, initial_location=[0, 0], zoom_start=10):
    # Create a map centered at the initial location
    map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
    m = folium.Map(location=map_center, zoom_start=zoom_start, tiles='openstreetmap')
    
    #Colours for Visualization
    amenity_colors = {
        'education': 'green',
        'finance': 'blue',
        'government offices': 'red',
        'grocery': 'orange',
        'health': 'magenta',
        'malls': 'yellow',
        'residential areas': 'brown',
        'security': 'gray',
        'transportation': 'lightblue',
        'others': 'black'
    }

    # Iterate over the nodes in the network
    for node, data in amenities_network.nodes(data=True):
        if data['amenity'] == amenity:
            # Check if the node has a geometry attribute
            if 'geometry' in data:
                # Get the geometry of the node
                geometry = data['geometry']

                # Check the geometry type and plot accordingly
                if geometry.geom_type == 'Point':
                    # Plot a marker for points    
                    #folium.Marker(location=[geometry.y, geometry.x], popup=f"{data['name']}").add_to(m)
                    continue
                elif geometry.geom_type in ['Polygon', 'MultiPolygon']:
                    # Plot polygons or multipolygons
                    color = amenity_colors[data.get('amenity')]
                    if geometry.geom_type == 'Polygon':
                        polygons = [geometry]
                    else:
                        polygons = geometry.geoms

                    for polygon in polygons:
                        coordinates = []
                        for point in polygon.exterior.coords:
                            coordinates.append([point[1], point[0]])
                        folium.Polygon(locations=coordinates, fill=True, color=color, fill_opacity=0.4).add_to(m)

    # Return the map
    return m